#**Imports**

##**Mount drive**
This line connect our colab with the Drive where we had the images

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**Enabling GPU Acceleration**
Colab notebooks run code on Google's cloud servers, allowing you to harness the power of Google hardware, including GPUs and TPUs, regardless of the power of your equipment.

This is good for us, because run model, proccess image ..., are stressful jobs for our hadware

On the other GPU's training are fasters than CPU's:
```
The time to convolve 32x7x7x3 filter over random 100x100x100x3 images.

CPU (s):
3.862475891000031
GPU (s):
0.10837535100017703
GPU speedup over CPU: 35x
```

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

##**Libraries That We Will Need**
``` 
numpy
 [NumPy is a library from Python that supports creating large multidimensional
  arrays and vectors, along with a large collection of high-level mathematical
  functions to operate on them]
os
  [This module provides a portable way of using operating system dependent 
  functionality]
pickle
  [module implements binary protocols for serializing and de-serializing a 
  Python object structure, in this project we user pickle to save the models]
tensorflow
  [TensorFlow is an open source library for machine learning developed by 
  Google, we use tensorflow to enabling the GPU Accelerator and Keras libraries]
keras
  [Keras is an Open Source Neural Networks library written in Python. It is 
  able to run on TensorFlow, Microsoft Cognitive Toolkit or Theano. It is
  specially designed to allow experimentation in more or less a short time 
  with Deep Learning networks] 
```

In [ ]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import os
import pickle
#Tensorflow Keras
from tensorflow.keras import layers,models,optimizers,preprocessing, applications
from keras.models import Sequential #libreria necesaria para apilar o alinear una serie de capas con la que crearemos las redes neuronales
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D #capas que usaremos
from tensorflow.keras.optimizers import Adam #optimizador necesario para compilar nuestro modelo en este caso usaremos Adam

#**Proccess Images**

##**Load Images**
**HIPERPARAMETERS**
```
main_dir: String, Image route, there are on my drive.
classes: Array of String, this array contain the posibles label of the image, 
         we had 4 classes.
resoultion: Number Int, We use this param to resize the image to 128x128.
epochs: Number Int, an epoch is when an ENTIRE dataset is passed forward and    
        backward through the neural network only ONCE, we put 50 epochs
batch_size: Number Int, Number of training examples present in a single batch, 
        we put 20.
```

In [ ]:
main_dir = "/content/drive/MyDrive/tfg/tumor-classificator/Total"
classes = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]
resolution = 128
epochs=50
batch_size=20

#Proccess and save images
def load_images():
  
  X_colored = []
  X_grayscale = []
  y = []
    
  for label, actual_class in enumerate(classes):
    print(f"loading from Total class {actual_class}")
    sub_dir_path = os.path.join(main_dir, actual_class)
    for image_name in os.listdir(sub_dir_path):
      image_path = os.path.join(sub_dir_path, image_name)
      #save images as rgb mode and resize to 128x128
      image_colored = preprocessing.image.load_img(image_path, color_mode="rgb", target_size=(resolution, resolution))
      #save images as grayscale mode and resize to 128x128
      image_grayscale = preprocessing.image.load_img(image_path, color_mode="grayscale", target_size=(resolution, resolution))
      #save in respectives arrays in order
      X_colored.append(preprocessing.image.img_to_array(image_colored))
      X_grayscale.append(preprocessing.image.img_to_array(image_grayscale))
      #save labels per image 
      y.append(label)

  #normalize data
  X_colored = np.array(X_colored) / 255.0
  X_grayscale = np.array(X_grayscale) / 255.0
  y = np.array(y)
    
  return X_colored, X_grayscale, y

In [ ]:
X_colored, X_grayscale, y = load_images()

##**Convert To One-Hot Encoding**

In [ ]:
from keras.utils.np_utils import to_categorical 
y = to_categorical(y, num_classes = 4)

##**Split The Images**

In [ ]:
from sklearn.model_selection import train_test_split
#divide images in 3 batchs train, validation and test by colors
X_train_rgb, X_val_rgb, y_train_rgb, y_val_rgb = train_test_split(X_colored, y, test_size = 0.2, random_state=132)
X_train_rgb, X_test_rgb, y_train_rgb, y_test_rgb = train_test_split(X_train_rgb, y_train_rgb, test_size = 0.1, random_state=132)

X_train_gs, X_val_gs, y_train_gs, y_val_gs = train_test_split(X_grayscale, y, test_size = 0.2, random_state=132)
X_train_gs, X_test_gs, y_train_gs, y_test_gs = train_test_split(X_train_gs, y_train_gs, test_size = 0.1, random_state=132)

#**Create Models**

##**Custom**

In [ ]:
def create_custom_models(input_shape):
  model = Sequential()

  model.add(Conv2D(filters = 64, kernel_size = (2,2),
            padding = 'Same', activation ='relu', input_shape = input_shape))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(filters = 128, kernel_size = (2,2),
            padding = 'Same', activation ='relu'))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(filters = 256, kernel_size = (2,2),
            padding = 'Same', activation ='relu'))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(filters = 128, kernel_size = (2,2),
            padding = 'Same', activation ='relu'))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(filters = 512, kernel_size = (2,2),
            padding = 'Same', activation ='relu'))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Flatten())

  model.add(Dense(1024, activation = "relu"))
  model.add(Dropout(0.5))

  model.add(Dense(4, activation = "softmax"))

  model.compile(optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
                loss = "categorical_crossentropy", metrics=["accuracy"])
  return model

##**VGGs**
```
tf.keras.applications.VGG1X(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
```

In [ ]:
#Creamos una red neuronal con base VGG16
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
def create_vgg_model(model_version, input_shape,transfer_learning=False):
  
  if model_version != 16 and model_version != 19:
    print("Error Vesion Model Incorrect")
    return null
  
  if transfer_learning and input_shape[-1] == 3:
    if model_version == 16:
      vgg_conv = VGG16(weights= "imagenet" , include_top=False, input_shape=input_shape)
    else:
      vgg_conv = VGG19(weights= "imagenet" , include_top=False, input_shape=input_shape)
    vgg_conv.trainable = False
    
    model = Sequential()
    model.add(vgg_conv)

    # Y le añadimos las capas que queramos
    model.add(Flatten()) 
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(4, activation='softmax'))

  else:
    if model_version == 16:
      model = VGG16(include_top=True, weights=None,  input_shape=input_shape, classes=4)
    else:
      model = VGG19(include_top=True, weights=None,  input_shape=input_shape, classes=4)

  #compilamos el modelo con el optimizador adam y el factor de perdidas categorical_crossentropy
  model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999), metrics=['accuracy'])
  return model

##**Xception**
```
tf.keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
```

In [ ]:
def create_Xception_model(input_shape,transfer_learning=False):
  if transfer_learning and input_shape[-1] == 3:
    Xception_model = tf.keras.applications.Xception(weights="imagenet", include_top=False, input_shape=input_shape)
    Xception_model.trainable = False
    model = Sequential()
  
    model.add(Xception_model)

    # Y le añadimos las capas que queramos
    model.add(Flatten()) 
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(4, activation='softmax')) 
  else:
    model = tf.keras.applications.Xception(weights=None, input_shape=input_shape,classes=4)
  model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999), metrics=['accuracy'])
  return model

##**ResNets**
```
tf.keras.applications.ResNetX(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    **kwargs
)
```

In [ ]:
def create_resnet_model(resnet_version, input_shape, transfer_learning=False):
  if transfer_learning and input_shape[-1] == 3:
    if resnet_version == 50:
      resnet_model = tf.keras.applications.ResNet50(
          weights="imagenet", include_top=False, input_shape=input_shape)
    elif resnet_version == 101:
      resnet_model = tf.keras.applications.ResNet101(
          weights="imagenet", include_top=False, input_shape=input_shape)
    else:
      resnet_model = tf.keras.applications.ResNet152(
          weights="imagenet", include_top=False, input_shape=input_shape)
    
    resnet_model.trainable = False
    model = Sequential()
  
    model.add(resnet_model)

    # Y le añadimos las capas que queramos
    model.add(Flatten()) 
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(4, activation='softmax')) 
  else:
    if resnet_version == 50:
      model = tf.keras.applications.ResNet50(
          weights=None, input_shape=input_shape, classes=4)
    elif resnet_version == 101:
      model = tf.keras.applications.ResNet101(
          weights=None, input_shape=input_shape, classes=4)
    else:
      model = tf.keras.applications.ResNet152(
          weights=None, input_shape=input_shape, classes=4)
      
  model.compile(optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
                loss = "categorical_crossentropy", metrics=["accuracy"])
  return model

##**ResNetV2s**
```
tf.keras.applications.ResNetXV2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
```

In [ ]:
from keras.applications import ResNet50V2, ResNet101V2, ResNet152V2

def create_resnetv2_model(resnet_version, input_shape, transfer_learning=False):
  if transfer_learning and input_shape[-1] == 3:
    if resnet_version == 50:
      resnetv2_model = ResNet50V2(weights="imagenet", include_top=False, input_shape=input_shape)
    elif resnet_version == 101:
      resnetv2_model = ResNet101V2(weights="imagenet", include_top=False, input_shape=input_shape)
    else:
      resnetv2_model = ResNet152V2(weights="imagenet", include_top=False, input_shape=input_shape)
    
    resnetv2_model.trainable = False
    model = Sequential()
  
    model.add(resnetv2_model)

    # Y le añadimos las capas que queramos
    model.add(Flatten()) 
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(4, activation='softmax')) 
  else:
    if resnet_version == 50:
      model = ResNet50V2(weights=None, input_shape=input_shape, classes=4)
    elif resnet_version == 101:
      model = ResNet101V2(weights=None, input_shape=input_shape, classes=4)
    else:
      model = ResNet152V2(weights=None, input_shape=input_shape, classes=4)
      
  model.compile(optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999), loss = "categorical_crossentropy", metrics=["accuracy"])
  return model

#Training Without Data Augmentation

##Training Custom Model

###Custom Model RGB

In [ ]:
modelCNN_rgb = create_custom_models(X_train_rgb.shape[1:])

In [ ]:
history_modelCNN_rgb = modelCNN_rgb.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 50s 43ms/step - loss: 1.3871 - accuracy: 0.3343 - val_loss: 1.2597 - val_accuracy: 0.4366
Epoch 2/50
118/118 [==============================] - 4s 30ms/step - loss: 1.0982 - accuracy: 0.4979 - val_loss: 0.9763 - val_accuracy: 0.5603
Epoch 3/50
118/118 [==============================] - 3s 30ms/step - loss: 0.9375 - accuracy: 0.5769 - val_loss: 0.7779 - val_accuracy: 0.6901
Epoch 4/50
118/118 [==============================] - 3s 30ms/step - loss: 0.8439 - accuracy: 0.6355 - val_loss: 0.7205 - val_accuracy: 0.6840
Epoch 5/50
118/118 [==============================] - 4s 30ms/step - loss: 0.7011 - accuracy: 0.6958 - val_loss: 0.7381 - val_accuracy: 0.6947
Epoch 6/50
118/118 [==============================] - 4s 30ms/step - loss: 0.6438 - accuracy: 0.7379 - val_loss: 0.5816 - val_accuracy: 0.7588
Epoch 7/50
118/118 [==============================] - 4s 30ms/step - loss: 0.5999 - accuracy: 0.7574 - val_loss: 0.5380 - val_accuracy: 0.783

###Custom Model GS

In [ ]:
modelCNN_gs = create_custom_models(X_train_gs.shape[1:])

In [ ]:
history_modelCNN_gs = modelCNN_gs.fit(X_train_gs, y_train_gs, batch_size=batch_size, epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 5s 33ms/step - loss: 1.3245 - accuracy: 0.3527 - val_loss: 0.9319 - val_accuracy: 0.6092
Epoch 2/50
118/118 [==============================] - 4s 31ms/step - loss: 0.9971 - accuracy: 0.5740 - val_loss: 0.8034 - val_accuracy: 0.6702
Epoch 3/50
118/118 [==============================] - 4s 31ms/step - loss: 0.8448 - accuracy: 0.6523 - val_loss: 0.6770 - val_accuracy: 0.7359
Epoch 4/50
118/118 [==============================] - 4s 31ms/step - loss: 0.6812 - accuracy: 0.7095 - val_loss: 0.5726 - val_accuracy: 0.7634
Epoch 5/50
118/118 [==============================] - 4s 31ms/step - loss: 0.5977 - accuracy: 0.7505 - val_loss: 0.6053 - val_accuracy: 0.7389
Epoch 6/50
118/118 [==============================] - 4s 31ms/step - loss: 0.5898 - accuracy: 0.7607 - val_loss: 0.5706 - val_accuracy: 0.7481
Epoch 7/50
118/118 [==============================] - 4s 31ms/step - loss: 0.4635 - accuracy: 0.8195 - val_loss: 0.4589 - val_accuracy: 0.7985

##Training VGG16

###VGG16 RGB


In [ ]:
modelVGG16_rgb = create_vgg_model(16,X_train_rgb.shape[1:])

In [ ]:
history_modelVGG16_rgb = modelVGG16_rgb.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 17s 134ms/step - loss: 1.3672 - accuracy: 0.2836 - val_loss: 1.3851 - val_accuracy: 0.2672
Epoch 2/50
118/118 [==============================] - 15s 129ms/step - loss: 1.3606 - accuracy: 0.2773 - val_loss: 1.3923 - val_accuracy: 0.2672
Epoch 3/50
118/118 [==============================] - 15s 127ms/step - loss: 1.3510 - accuracy: 0.2822 - val_loss: 1.3843 - val_accuracy: 0.2672
Epoch 4/50
118/118 [==============================] - 15s 125ms/step - loss: 1.3494 - accuracy: 0.2796 - val_loss: 1.3794 - val_accuracy: 0.2672
Epoch 5/50
118/118 [==============================] - 15s 125ms/step - loss: 1.3506 - accuracy: 0.2870 - val_loss: 1.3845 - val_accuracy: 0.2687
Epoch 6/50
118/118 [==============================] - 15s 126ms/step - loss: 1.3615 - accuracy: 0.2891 - val_loss: 1.3824 - val_accuracy: 0.2672
Epoch 7/50
118/118 [==============================] - 15s 127ms/step - loss: 1.3477 - accuracy: 0.2965 - val_loss: 1.3838 - val_ac

###VGG16 GS

In [ ]:
modelVGG16_gs = create_vgg_model(16,X_train_gs.shape[1:])

In [ ]:
history_modelVGG16_gs = modelVGG16_gs.fit(X_train_gs, y_train_gs, batch_size=batch_size, epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 17s 133ms/step - loss: 1.3658 - accuracy: 0.2903 - val_loss: 1.3878 - val_accuracy: 0.2672
Epoch 2/50
118/118 [==============================] - 15s 129ms/step - loss: 1.3550 - accuracy: 0.2811 - val_loss: 1.3818 - val_accuracy: 0.2672
Epoch 3/50
118/118 [==============================] - 15s 129ms/step - loss: 1.3508 - accuracy: 0.2876 - val_loss: 1.3802 - val_accuracy: 0.2687
Epoch 4/50
118/118 [==============================] - 15s 129ms/step - loss: 1.3562 - accuracy: 0.2963 - val_loss: 1.3836 - val_accuracy: 0.2687
Epoch 5/50
118/118 [==============================] - 15s 129ms/step - loss: 1.3446 - accuracy: 0.2830 - val_loss: 1.3778 - val_accuracy: 0.2672
Epoch 6/50
118/118 [==============================] - 15s 129ms/step - loss: 1.3546 - accuracy: 0.2759 - val_loss: 1.3939 - val_accuracy: 0.2672
Epoch 7/50
118/118 [==============================] - 15s 129ms/step - loss: 1.3536 - accuracy: 0.3076 - val_loss: 1.3810 - val_ac

###VGG16 RGB With TFL

In [ ]:
modelVGG16_tfl = create_vgg_model(16,X_train_rgb.shape[1:], True)

In [ ]:
history_modelVGG16_tfl = modelVGG16_tfl.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 7s 51ms/step - loss: 2.3429 - accuracy: 0.5188 - val_loss: 0.5019 - val_accuracy: 0.8137
Epoch 2/50
118/118 [==============================] - 6s 49ms/step - loss: 0.5051 - accuracy: 0.8061 - val_loss: 0.5182 - val_accuracy: 0.7954
Epoch 3/50
118/118 [==============================] - 6s 49ms/step - loss: 0.3471 - accuracy: 0.8623 - val_loss: 0.6735 - val_accuracy: 0.7542
Epoch 4/50
118/118 [==============================] - 6s 49ms/step - loss: 0.3285 - accuracy: 0.8726 - val_loss: 0.4028 - val_accuracy: 0.8504
Epoch 5/50
118/118 [==============================] - 6s 49ms/step - loss: 0.2910 - accuracy: 0.8901 - val_loss: 0.3268 - val_accuracy: 0.8840
Epoch 6/50
118/118 [==============================] - 6s 49ms/step - loss: 0.2334 - accuracy: 0.9117 - val_loss: 0.3478 - val_accuracy: 0.8931
Epoch 7/50
118/118 [==============================] - 6s 49ms/step - loss: 0.2190 - accuracy: 0.9207 - val_loss: 0.3892 - val_accuracy: 0.8641

##Training VGG19

###VGG19 RGB

In [ ]:
modelVGG19_rgb = create_vgg_model(19,X_train_rgb.shape[1:])

In [ ]:
history_modelVGG19_rgb = modelVGG19_rgb.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 19s 151ms/step - loss: 1.3584 - accuracy: 0.2742 - val_loss: 1.3767 - val_accuracy: 0.2763
Epoch 2/50
118/118 [==============================] - 17s 148ms/step - loss: 1.3621 - accuracy: 0.2572 - val_loss: 1.4069 - val_accuracy: 0.2672
Epoch 3/50
118/118 [==============================] - 17s 148ms/step - loss: 1.3582 - accuracy: 0.2991 - val_loss: 1.3881 - val_accuracy: 0.2672
Epoch 4/50
118/118 [==============================] - 17s 148ms/step - loss: 1.3604 - accuracy: 0.2931 - val_loss: 1.3874 - val_accuracy: 0.2687
Epoch 5/50
118/118 [==============================] - 17s 148ms/step - loss: 1.3515 - accuracy: 0.2873 - val_loss: 1.3800 - val_accuracy: 0.2672
Epoch 6/50
118/118 [==============================] - 17s 148ms/step - loss: 1.3567 - accuracy: 0.2792 - val_loss: 1.3907 - val_accuracy: 0.2672
Epoch 7/50
118/118 [==============================] - 17s 148ms/step - loss: 1.3588 - accuracy: 0.2881 - val_loss: 1.3948 - val_ac

###VGG19 GS

In [ ]:
modelVGG19_gs = create_vgg_model(19,X_train_gs.shape[1:])

In [ ]:
history_modelVGG19_gs = modelVGG19_gs.fit(X_train_gs, y_train_gs, batch_size=batch_size, epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 19s 152ms/step - loss: 1.3712 - accuracy: 0.2810 - val_loss: 1.3981 - val_accuracy: 0.2672
Epoch 2/50
118/118 [==============================] - 17s 148ms/step - loss: 1.3537 - accuracy: 0.2950 - val_loss: 1.3868 - val_accuracy: 0.2672
Epoch 3/50
118/118 [==============================] - 18s 148ms/step - loss: 1.3497 - accuracy: 0.2931 - val_loss: 1.3862 - val_accuracy: 0.2672
Epoch 4/50
118/118 [==============================] - 17s 148ms/step - loss: 1.3579 - accuracy: 0.2965 - val_loss: 1.3880 - val_accuracy: 0.2672
Epoch 5/50
118/118 [==============================] - 17s 148ms/step - loss: 1.3579 - accuracy: 0.3026 - val_loss: 1.3792 - val_accuracy: 0.2763
Epoch 6/50
118/118 [==============================] - 17s 148ms/step - loss: 1.3502 - accuracy: 0.2676 - val_loss: 1.3816 - val_accuracy: 0.2687
Epoch 7/50
118/118 [==============================] - 17s 148ms/step - loss: 1.3575 - accuracy: 0.2639 - val_loss: 1.3826 - val_ac

###VGG19 RGB With TFL

In [ ]:
modelVGG19_tfl = create_vgg_model(19,X_train_rgb.shape[1:],True)

In [ ]:
history_modelVGG19_tfl = modelVGG19_tfl.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 8s 61ms/step - loss: 3.3892 - accuracy: 0.4601 - val_loss: 0.5966 - val_accuracy: 0.7481
Epoch 2/50
118/118 [==============================] - 7s 59ms/step - loss: 0.5768 - accuracy: 0.7680 - val_loss: 0.5263 - val_accuracy: 0.8061
Epoch 3/50
118/118 [==============================] - 7s 59ms/step - loss: 0.4854 - accuracy: 0.8090 - val_loss: 0.5029 - val_accuracy: 0.8321
Epoch 4/50
118/118 [==============================] - 7s 59ms/step - loss: 0.3726 - accuracy: 0.8500 - val_loss: 0.4091 - val_accuracy: 0.8565
Epoch 5/50
118/118 [==============================] - 7s 59ms/step - loss: 0.3766 - accuracy: 0.8506 - val_loss: 0.5865 - val_accuracy: 0.7969
Epoch 6/50
118/118 [==============================] - 7s 59ms/step - loss: 0.4225 - accuracy: 0.8415 - val_loss: 0.4012 - val_accuracy: 0.8443
Epoch 7/50
118/118 [==============================] - 7s 59ms/step - loss: 0.3049 - accuracy: 0.8803 - val_loss: 0.3336 - val_accuracy: 0.8870

##Training Xception

###Xception RGB

In [ ]:
modelXception_rgb = create_Xception_model(X_train_rgb.shape[1:])

In [ ]:
history_modelXception_rgb = modelXception_rgb.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 72s 318ms/step - loss: 1.1886 - accuracy: 0.5276 - val_loss: 1.3863 - val_accuracy: 0.2763
Epoch 2/50
118/118 [==============================] - 35s 299ms/step - loss: 0.6221 - accuracy: 0.7597 - val_loss: 1.4031 - val_accuracy: 0.1878
Epoch 3/50
118/118 [==============================] - 35s 299ms/step - loss: 0.4488 - accuracy: 0.8239 - val_loss: 1.3842 - val_accuracy: 0.3145
Epoch 4/50
118/118 [==============================] - 35s 298ms/step - loss: 0.3818 - accuracy: 0.8646 - val_loss: 1.1427 - val_accuracy: 0.4641
Epoch 5/50
118/118 [==============================] - 35s 298ms/step - loss: 0.2876 - accuracy: 0.8937 - val_loss: 0.4978 - val_accuracy: 0.8275
Epoch 6/50
118/118 [==============================] - 35s 299ms/step - loss: 0.2492 - accuracy: 0.9112 - val_loss: 0.4151 - val_accuracy: 0.8534
Epoch 7/50
118/118 [==============================] - 35s 299ms/step - loss: 0.2375 - accuracy: 0.9148 - val_loss: 3.2910 - val_ac

###Xception GS

In [ ]:
modelXception_gs = create_Xception_model(X_train_gs.shape[1:])

In [ ]:
history_modelXception_gs = modelXception_gs.fit(X_train_gs, y_train_gs, batch_size=batch_size, epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 41s 304ms/step - loss: 1.0815 - accuracy: 0.5534 - val_loss: 1.3851 - val_accuracy: 0.2763
Epoch 2/50
118/118 [==============================] - 35s 297ms/step - loss: 0.6507 - accuracy: 0.7635 - val_loss: 1.3886 - val_accuracy: 0.1878
Epoch 3/50
118/118 [==============================] - 35s 296ms/step - loss: 0.5171 - accuracy: 0.8114 - val_loss: 1.4023 - val_accuracy: 0.1878
Epoch 4/50
118/118 [==============================] - 35s 296ms/step - loss: 0.2544 - accuracy: 0.9088 - val_loss: 1.2516 - val_accuracy: 0.5130
Epoch 5/50
118/118 [==============================] - 35s 298ms/step - loss: 0.2417 - accuracy: 0.9180 - val_loss: 0.5754 - val_accuracy: 0.7557
Epoch 6/50
118/118 [==============================] - 35s 297ms/step - loss: 0.1954 - accuracy: 0.9312 - val_loss: 0.4815 - val_accuracy: 0.8458
Epoch 7/50
118/118 [==============================] - 35s 297ms/step - loss: 0.1702 - accuracy: 0.9385 - val_loss: 0.5408 - val_ac

###Xception RGB With TFL

In [ ]:
modelXception_tfl = create_Xception_model(X_train_rgb.shape[1:], True)

In [ ]:
history_modelXception_tfl = modelXception_tfl.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 44s 332ms/step - loss: 2.1118 - accuracy: 0.3214 - val_loss: 1.2237 - val_accuracy: 0.3985
Epoch 2/50
118/118 [==============================] - 39s 327ms/step - loss: 1.0497 - accuracy: 0.5059 - val_loss: 1.1448 - val_accuracy: 0.6244
Epoch 3/50
118/118 [==============================] - 39s 327ms/step - loss: 0.7656 - accuracy: 0.6562 - val_loss: 0.7166 - val_accuracy: 0.7481
Epoch 4/50
118/118 [==============================] - 39s 328ms/step - loss: 0.4401 - accuracy: 0.8580 - val_loss: 14.1155 - val_accuracy: 0.4794
Epoch 5/50
118/118 [==============================] - 39s 328ms/step - loss: 0.3691 - accuracy: 0.8853 - val_loss: 0.2595 - val_accuracy: 0.9252
Epoch 6/50
118/118 [==============================] - 39s 327ms/step - loss: 0.1873 - accuracy: 0.9493 - val_loss: 0.4482 - val_accuracy: 0.8183
Epoch 7/50
118/118 [==============================] - 39s 327ms/step - loss: 0.1349 - accuracy: 0.9639 - val_loss: 0.4750 - val_a

##Training ResNet

### ResNet50 RGB

In [ ]:
modelResnet50_rgb = create_resnet_model(50,X_train_rgb.shape[1:])

In [ ]:
history_modelResnet50_rgb = modelResnet50_rgb.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 66s 250ms/step - loss: 2.1029 - accuracy: 0.5064 - val_loss: 2.0674 - val_accuracy: 0.2824
Epoch 2/50
118/118 [==============================] - 26s 221ms/step - loss: 0.8491 - accuracy: 0.6942 - val_loss: 2.5032 - val_accuracy: 0.2397
Epoch 3/50
118/118 [==============================] - 26s 221ms/step - loss: 0.6271 - accuracy: 0.7681 - val_loss: 1.8338 - val_accuracy: 0.3740
Epoch 4/50
118/118 [==============================] - 26s 222ms/step - loss: 0.5528 - accuracy: 0.7886 - val_loss: 1.8339 - val_accuracy: 0.3817
Epoch 5/50
118/118 [==============================] - 26s 221ms/step - loss: 0.4896 - accuracy: 0.8132 - val_loss: 0.6807 - val_accuracy: 0.7405
Epoch 6/50
118/118 [==============================] - 26s 222ms/step - loss: 0.3699 - accuracy: 0.8659 - val_loss: 0.5851 - val_accuracy: 0.8214
Epoch 7/50
118/118 [==============================] - 26s 222ms/step - loss: 0.3067 - accuracy: 0.8916 - val_loss: 1.4021 - val_ac

###ResNet50 GS

In [ ]:
modelResnet50_gs = create_resnet_model(50,X_train_gs.shape[1:])

In [ ]:
history_modelResnet50_gs = modelResnet50_gs.fit(X_train_gs, y_train_gs, batch_size=batch_size, epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 33s 228ms/step - loss: 2.5566 - accuracy: 0.4979 - val_loss: 1.8564 - val_accuracy: 0.2687
Epoch 2/50
118/118 [==============================] - 26s 218ms/step - loss: 0.7473 - accuracy: 0.7097 - val_loss: 3.8205 - val_accuracy: 0.2901
Epoch 3/50
118/118 [==============================] - 26s 218ms/step - loss: 0.6231 - accuracy: 0.7755 - val_loss: 3.2894 - val_accuracy: 0.1878
Epoch 4/50
118/118 [==============================] - 26s 218ms/step - loss: 0.5862 - accuracy: 0.8115 - val_loss: 3.7927 - val_accuracy: 0.2992
Epoch 5/50
118/118 [==============================] - 26s 217ms/step - loss: 0.4507 - accuracy: 0.8465 - val_loss: 1.0407 - val_accuracy: 0.5802
Epoch 6/50
118/118 [==============================] - 26s 218ms/step - loss: 0.3125 - accuracy: 0.8851 - val_loss: 1.3686 - val_accuracy: 0.7130
Epoch 7/50
118/118 [==============================] - 26s 218ms/step - loss: 0.2635 - accuracy: 0.9054 - val_loss: 1.1014 - val_ac

###ResNet50 RGB With TFL

In [ ]:
modelResnet50_tfl = create_resnet_model(50,X_train_rgb.shape[1:],True)

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
history_modelResnet50_tfl = modelResnet50_tfl.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 47s 122ms/step - loss: 2.6581 - accuracy: 0.3897 - val_loss: 1.0995 - val_accuracy: 0.5695
Epoch 2/50
118/118 [==============================] - 12s 102ms/step - loss: 1.0804 - accuracy: 0.5345 - val_loss: 0.9021 - val_accuracy: 0.6336
Epoch 3/50
118/118 [==============================] - 12s 102ms/step - loss: 0.9712 - accuracy: 0.5718 - val_loss: 0.8296 - val_accuracy: 0.7053
Epoch 4/50
118/118 [==============================] - 12s 103ms/step - loss: 0.8795 - accuracy: 0.6248 - val_loss: 0.9201 - val_accuracy: 0.6214
Epoch 5/50
118/118 [==============================] - 12s 104ms/step - loss: 0.8601 - accuracy: 0.6440 - val_loss: 0.8813 - val_accuracy: 0.6305
Epoch 6/50
118/118 [==============================] - 12s 104ms/step - loss: 0.8169 - accuracy: 0.6503 - val_loss: 0.7212 - val_accuracy: 0.7115
Epoch 7/50
118/118 [==============================] - 12s 104ms/step - loss: 0.7920 - accuracy: 0.6657 - val_loss: 0.7272 - val_ac

###ResNet101 RGB

In [ ]:
modelResnet101_rgb = create_resnet_model(101,X_train_rgb.shape[1:])

In [ ]:
history_modelResnet101_rgb = modelResnet101_rgb.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 60s 405ms/step - loss: 2.0884 - accuracy: 0.5013 - val_loss: 1.9570 - val_accuracy: 0.2763
Epoch 2/50
118/118 [==============================] - 46s 389ms/step - loss: 0.9522 - accuracy: 0.6589 - val_loss: 2.4215 - val_accuracy: 0.3344
Epoch 3/50
118/118 [==============================] - 46s 388ms/step - loss: 0.6811 - accuracy: 0.7339 - val_loss: 1.7757 - val_accuracy: 0.3282
Epoch 4/50
118/118 [==============================] - 46s 387ms/step - loss: 0.6196 - accuracy: 0.7483 - val_loss: 1.3825 - val_accuracy: 0.5542
Epoch 5/50
118/118 [==============================] - 46s 388ms/step - loss: 0.5508 - accuracy: 0.7707 - val_loss: 1.2517 - val_accuracy: 0.6351
Epoch 6/50
118/118 [==============================] - 46s 388ms/step - loss: 0.4652 - accuracy: 0.8386 - val_loss: 4.4101 - val_accuracy: 0.6794
Epoch 7/50
118/118 [==============================] - 46s 388ms/step - loss: 0.3641 - accuracy: 0.8780 - val_loss: 2.7389 - val_ac

###ResNet101 GS

In [ ]:
modelResnet101_gs = create_resnet_model(101,X_train_gs.shape[1:])

In [ ]:
history_modelResnet101_gs = modelResnet101_gs.fit(X_train_gs, y_train_gs, batch_size=batch_size, epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 64s 196ms/step - loss: 1.5170 - accuracy: 0.5070 - val_loss: 2.6855 - val_accuracy: 0.2763
Epoch 2/50
118/118 [==============================] - 19s 161ms/step - loss: 0.9750 - accuracy: 0.6445 - val_loss: 3.0250 - val_accuracy: 0.2748
Epoch 3/50
118/118 [==============================] - 19s 158ms/step - loss: 0.7778 - accuracy: 0.6988 - val_loss: 1.9279 - val_accuracy: 0.2779
Epoch 4/50
118/118 [==============================] - 19s 158ms/step - loss: 0.6614 - accuracy: 0.7425 - val_loss: 1.6401 - val_accuracy: 0.3557
Epoch 5/50
118/118 [==============================] - 19s 160ms/step - loss: 0.5795 - accuracy: 0.7798 - val_loss: 2.1343 - val_accuracy: 0.4031
Epoch 6/50
118/118 [==============================] - 20s 171ms/step - loss: 0.4772 - accuracy: 0.8231 - val_loss: 1.1920 - val_accuracy: 0.5221
Epoch 7/50
118/118 [==============================] - 19s 160ms/step - loss: 0.4129 - accuracy: 0.8498 - val_loss: 0.9269 - val_ac

###ResNet101 RGB With TFL

In [ ]:
modelResnet101_tfl = create_resnet_model(101,X_train_rgb.shape[1:],True)

171450368/171446536 [==============================] - 4s 0us/step


In [ ]:
history_modelResnet101_tfl = modelResnet101_tfl.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 27s 177ms/step - loss: 2.3523 - accuracy: 0.3340 - val_loss: 0.9916 - val_accuracy: 0.5695
Epoch 2/50
118/118 [==============================] - 19s 160ms/step - loss: 0.9634 - accuracy: 0.5999 - val_loss: 0.9104 - val_accuracy: 0.5985
Epoch 3/50
118/118 [==============================] - 19s 160ms/step - loss: 0.8680 - accuracy: 0.6313 - val_loss: 0.8473 - val_accuracy: 0.6198
Epoch 4/50
118/118 [==============================] - 19s 160ms/step - loss: 0.8103 - accuracy: 0.6761 - val_loss: 0.7520 - val_accuracy: 0.6855
Epoch 5/50
118/118 [==============================] - 19s 160ms/step - loss: 0.7768 - accuracy: 0.6910 - val_loss: 0.7211 - val_accuracy: 0.7191
Epoch 6/50
118/118 [==============================] - 19s 160ms/step - loss: 0.7603 - accuracy: 0.6846 - val_loss: 0.7680 - val_accuracy: 0.6931
Epoch 7/50
118/118 [==============================] - 19s 160ms/step - loss: 0.7035 - accuracy: 0.6893 - val_loss: 0.6558 - val_ac

###ResNet152 RGB

In [ ]:
modelResnet152_rgb = create_resnet_model(152,X_train_rgb.shape[1:])

In [ ]:
history_modelResnet152_rgb = modelResnet152_rgb.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 75s 242ms/step - loss: 2.4100 - accuracy: 0.3409 - val_loss: 1.8966 - val_accuracy: 0.2687
Epoch 2/50
118/118 [==============================] - 25s 212ms/step - loss: 1.1044 - accuracy: 0.5806 - val_loss: 1.5514 - val_accuracy: 0.3038
Epoch 3/50
118/118 [==============================] - 25s 215ms/step - loss: 0.9546 - accuracy: 0.6388 - val_loss: 1.5411 - val_accuracy: 0.3771
Epoch 4/50
118/118 [==============================] - 26s 217ms/step - loss: 0.7871 - accuracy: 0.6800 - val_loss: 1.2768 - val_accuracy: 0.4733
Epoch 5/50
118/118 [==============================] - 26s 218ms/step - loss: 0.6692 - accuracy: 0.7362 - val_loss: 3.7066 - val_accuracy: 0.4962
Epoch 6/50
118/118 [==============================] - 26s 219ms/step - loss: 0.6625 - accuracy: 0.7434 - val_loss: 2.7766 - val_accuracy: 0.4947
Epoch 7/50
118/118 [==============================] - 26s 220ms/step - loss: 0.5947 - accuracy: 0.7746 - val_loss: 2.7100 - val_ac

###ResNet152 GS

In [ ]:
modelResnet152_gs = create_resnet_model(152,X_train_gs.shape[1:])

In [ ]:
history_modelResnet152_gs = modelResnet152_gs.fit(X_train_gs, y_train_gs, batch_size=batch_size, epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 42s 240ms/step - loss: 2.6168 - accuracy: 0.3892 - val_loss: 2.6105 - val_accuracy: 0.2687
Epoch 2/50
118/118 [==============================] - 26s 218ms/step - loss: 1.0801 - accuracy: 0.5826 - val_loss: 2.4967 - val_accuracy: 0.2565
Epoch 3/50
118/118 [==============================] - 26s 219ms/step - loss: 0.7936 - accuracy: 0.7002 - val_loss: 3.6857 - val_accuracy: 0.2687
Epoch 4/50
118/118 [==============================] - 26s 220ms/step - loss: 0.7205 - accuracy: 0.7271 - val_loss: 2.4323 - val_accuracy: 0.3328
Epoch 5/50
118/118 [==============================] - 26s 220ms/step - loss: 0.6495 - accuracy: 0.7524 - val_loss: 2.0148 - val_accuracy: 0.3985
Epoch 6/50
118/118 [==============================] - 26s 220ms/step - loss: 0.5190 - accuracy: 0.8030 - val_loss: 0.8678 - val_accuracy: 0.7023
Epoch 7/50
118/118 [==============================] - 26s 221ms/step - loss: 0.4649 - accuracy: 0.8193 - val_loss: 17.0255 - val_a

###ResNet152 RGB With TFL

In [ ]:
modelResnet152_tfl = create_resnet_model(152,X_train_rgb.shape[1:],True)

234700800/234698864 [==============================] - 12s 0us/step


In [ ]:
history_modelResnet152_tfl = modelResnet152_tfl.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 38s 240ms/step - loss: 2.0955 - accuracy: 0.4070 - val_loss: 1.0972 - val_accuracy: 0.5649
Epoch 2/50
118/118 [==============================] - 25s 213ms/step - loss: 0.9986 - accuracy: 0.5681 - val_loss: 0.8639 - val_accuracy: 0.6687
Epoch 3/50
118/118 [==============================] - 25s 214ms/step - loss: 0.8942 - accuracy: 0.6179 - val_loss: 0.8712 - val_accuracy: 0.6763
Epoch 4/50
118/118 [==============================] - 25s 214ms/step - loss: 0.8228 - accuracy: 0.6507 - val_loss: 0.8126 - val_accuracy: 0.6489
Epoch 5/50
118/118 [==============================] - 25s 215ms/step - loss: 0.7941 - accuracy: 0.6732 - val_loss: 0.8091 - val_accuracy: 0.6931
Epoch 6/50
118/118 [==============================] - 25s 214ms/step - loss: 0.8023 - accuracy: 0.6864 - val_loss: 0.7010 - val_accuracy: 0.7313
Epoch 7/50
118/118 [==============================] - 25s 214ms/step - loss: 0.7307 - accuracy: 0.7107 - val_loss: 0.8675 - val_ac

##Training ResNetV2

###ResNet50V2 RGB

In [ ]:
modelResnet50V2_rgb = create_resnetv2_model(50,X_train_rgb.shape[1:])

In [ ]:
history_modelResnet50V2_rgb = modelResnet50V2_rgb.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 17s 102ms/step - loss: 1.4714 - accuracy: 0.5371 - val_loss: 1.7915 - val_accuracy: 0.3282
Epoch 2/50
118/118 [==============================] - 10s 86ms/step - loss: 0.7591 - accuracy: 0.6815 - val_loss: 3.3619 - val_accuracy: 0.2824
Epoch 3/50
118/118 [==============================] - 10s 87ms/step - loss: 0.6514 - accuracy: 0.7600 - val_loss: 1.0565 - val_accuracy: 0.5878
Epoch 4/50
118/118 [==============================] - 10s 88ms/step - loss: 0.4630 - accuracy: 0.8165 - val_loss: 2.7358 - val_accuracy: 0.4916
Epoch 5/50
118/118 [==============================] - 10s 88ms/step - loss: 0.3957 - accuracy: 0.8461 - val_loss: 1.1343 - val_accuracy: 0.6061
Epoch 6/50
118/118 [==============================] - 10s 88ms/step - loss: 0.3087 - accuracy: 0.8873 - val_loss: 1.2149 - val_accuracy: 0.6061
Epoch 7/50
118/118 [==============================] - 10s 89ms/step - loss: 0.1900 - accuracy: 0.9442 - val_loss: 2.1061 - val_accuracy

###ResNet50V2 GS

In [ ]:
modelResnet50V2_gs = create_resnetv2_model(50,X_train_gs.shape[1:])

In [ ]:
history_modelResnet50V2_gs = modelResnet50V2_gs.fit(X_train_gs, y_train_gs, batch_size=batch_size, epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 15s 94ms/step - loss: 1.3017 - accuracy: 0.5293 - val_loss: 4.8743 - val_accuracy: 0.2763
Epoch 2/50
118/118 [==============================] - 10s 88ms/step - loss: 0.7735 - accuracy: 0.6916 - val_loss: 4.1203 - val_accuracy: 0.2763
Epoch 3/50
118/118 [==============================] - 10s 88ms/step - loss: 0.6134 - accuracy: 0.7683 - val_loss: 2.6956 - val_accuracy: 0.2504
Epoch 4/50
118/118 [==============================] - 10s 88ms/step - loss: 0.4944 - accuracy: 0.8026 - val_loss: 1.3632 - val_accuracy: 0.4626
Epoch 5/50
118/118 [==============================] - 10s 88ms/step - loss: 0.3767 - accuracy: 0.8581 - val_loss: 1.3442 - val_accuracy: 0.5145
Epoch 6/50
118/118 [==============================] - 10s 88ms/step - loss: 0.3589 - accuracy: 0.8684 - val_loss: 0.6405 - val_accuracy: 0.8107
Epoch 7/50
118/118 [==============================] - 10s 88ms/step - loss: 0.2435 - accuracy: 0.9133 - val_loss: 0.7959 - val_accuracy:

###ResNet50V2 RGB With TFL

In [ ]:
modelResnet50V2_tfl = create_resnetv2_model(50,X_train_rgb.shape[1:],True)

94674944/94668760 [==============================] - 2s 0us/step


In [ ]:
history_modelResnet50V2_tfl = modelResnet50V2_tfl.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 17s 111ms/step - loss: 24.7389 - accuracy: 0.6024 - val_loss: 2.4253 - val_accuracy: 0.7573
Epoch 2/50
118/118 [==============================] - 11s 95ms/step - loss: 1.1766 - accuracy: 0.8352 - val_loss: 0.7016 - val_accuracy: 0.8397
Epoch 3/50
118/118 [==============================] - 11s 96ms/step - loss: 0.2498 - accuracy: 0.9140 - val_loss: 0.4732 - val_accuracy: 0.8672
Epoch 4/50
118/118 [==============================] - 11s 95ms/step - loss: 0.1628 - accuracy: 0.9471 - val_loss: 0.7773 - val_accuracy: 0.8489
Epoch 5/50
118/118 [==============================] - 11s 96ms/step - loss: 0.2254 - accuracy: 0.9521 - val_loss: 0.7099 - val_accuracy: 0.8702
Epoch 6/50
118/118 [==============================] - 11s 96ms/step - loss: 0.2154 - accuracy: 0.9552 - val_loss: 0.6458 - val_accuracy: 0.8855
Epoch 7/50
118/118 [==============================] - 11s 96ms/step - loss: 0.1268 - accuracy: 0.9628 - val_loss: 0.8055 - val_accurac

###ResNet101V2 RGB

In [ ]:
modelResnet101V2_rgb = create_resnetv2_model(101,X_train_rgb.shape[1:])

In [ ]:
history_modelResnet101V2_rgb = modelResnet101V2_rgb.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 29s 164ms/step - loss: 1.5738 - accuracy: 0.4805 - val_loss: 1.7696 - val_accuracy: 0.3298
Epoch 2/50
118/118 [==============================] - 18s 153ms/step - loss: 0.8379 - accuracy: 0.6514 - val_loss: 1.5939 - val_accuracy: 0.5099
Epoch 3/50
118/118 [==============================] - 18s 153ms/step - loss: 0.7409 - accuracy: 0.6981 - val_loss: 1.9052 - val_accuracy: 0.3603
Epoch 4/50
118/118 [==============================] - 18s 155ms/step - loss: 0.5940 - accuracy: 0.7727 - val_loss: 7.6741 - val_accuracy: 0.6046
Epoch 5/50
118/118 [==============================] - 18s 155ms/step - loss: 0.5218 - accuracy: 0.8059 - val_loss: 2.0784 - val_accuracy: 0.4901
Epoch 6/50
118/118 [==============================] - 18s 156ms/step - loss: 0.4138 - accuracy: 0.8399 - val_loss: 0.6322 - val_accuracy: 0.7328
Epoch 7/50
118/118 [==============================] - 19s 157ms/step - loss: 0.3382 - accuracy: 0.8774 - val_loss: 1.8304 - val_ac

###ResNet101V2 GS

In [ ]:
modelResnet101V2_gs = create_resnetv2_model(101,X_train_gs.shape[1:])

In [ ]:
history_modelResnet101V2_gs = modelResnet101V2_gs.fit(X_train_gs, y_train_gs, batch_size=batch_size, epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 31s 158ms/step - loss: 1.3476 - accuracy: 0.4913 - val_loss: 2.3165 - val_accuracy: 0.1878
Epoch 2/50
118/118 [==============================] - 17s 147ms/step - loss: 0.7937 - accuracy: 0.6755 - val_loss: 2.8813 - val_accuracy: 0.2916
Epoch 3/50
118/118 [==============================] - 18s 149ms/step - loss: 0.6546 - accuracy: 0.7198 - val_loss: 2.6529 - val_accuracy: 0.3527
Epoch 4/50
118/118 [==============================] - 18s 150ms/step - loss: 0.5113 - accuracy: 0.7881 - val_loss: 1.5344 - val_accuracy: 0.5618
Epoch 5/50
118/118 [==============================] - 18s 151ms/step - loss: 0.4519 - accuracy: 0.8214 - val_loss: 0.7132 - val_accuracy: 0.7328
Epoch 6/50
118/118 [==============================] - 18s 152ms/step - loss: 0.3594 - accuracy: 0.8563 - val_loss: 1.4206 - val_accuracy: 0.5649
Epoch 7/50
118/118 [==============================] - 18s 152ms/step - loss: 0.3242 - accuracy: 0.8808 - val_loss: 0.7994 - val_ac

###ResNet101V2 RGB With TFL

In [ ]:
modelResnet101V2_tfl = create_resnetv2_model(101,X_train_rgb.shape[1:],True)

171319296/171317808 [==============================] - 3s 0us/step


In [ ]:
history_modelResnet101V2_tfl = modelResnet101V2_tfl.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 31s 170ms/step - loss: 20.7154 - accuracy: 0.5840 - val_loss: 1.6242 - val_accuracy: 0.8198
Epoch 2/50
118/118 [==============================] - 18s 153ms/step - loss: 0.9118 - accuracy: 0.8609 - val_loss: 0.8391 - val_accuracy: 0.8595
Epoch 3/50
118/118 [==============================] - 18s 153ms/step - loss: 0.4441 - accuracy: 0.9100 - val_loss: 0.9325 - val_accuracy: 0.8473
Epoch 4/50
118/118 [==============================] - 18s 153ms/step - loss: 0.3071 - accuracy: 0.9289 - val_loss: 0.6900 - val_accuracy: 0.8626
Epoch 5/50
118/118 [==============================] - 18s 153ms/step - loss: 0.1441 - accuracy: 0.9658 - val_loss: 0.8115 - val_accuracy: 0.8718
Epoch 6/50
118/118 [==============================] - 18s 154ms/step - loss: 0.1944 - accuracy: 0.9476 - val_loss: 1.3576 - val_accuracy: 0.8565
Epoch 7/50
118/118 [==============================] - 18s 154ms/step - loss: 0.2355 - accuracy: 0.9505 - val_loss: 0.9607 - val_a

###ResNet152V2 RGB

In [ ]:
modelResnet152V2_rgb = create_resnetv2_model(152,X_train_rgb.shape[1:])

In [ ]:
history_modelResnet152V2_rgb = modelResnet152V2_rgb.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 41s 236ms/step - loss: 1.3627 - accuracy: 0.4712 - val_loss: 1.8150 - val_accuracy: 0.1053
Epoch 2/50
118/118 [==============================] - 26s 219ms/step - loss: 0.9226 - accuracy: 0.6130 - val_loss: 1.9842 - val_accuracy: 0.2595
Epoch 3/50
118/118 [==============================] - 26s 222ms/step - loss: 0.7331 - accuracy: 0.7063 - val_loss: 1.5257 - val_accuracy: 0.2885
Epoch 4/50
118/118 [==============================] - 26s 224ms/step - loss: 0.6204 - accuracy: 0.7581 - val_loss: 5.9772 - val_accuracy: 0.3786
Epoch 5/50
118/118 [==============================] - 26s 224ms/step - loss: 0.6671 - accuracy: 0.7392 - val_loss: 1.0292 - val_accuracy: 0.5908
Epoch 6/50
118/118 [==============================] - 27s 225ms/step - loss: 0.5179 - accuracy: 0.7945 - val_loss: 0.6718 - val_accuracy: 0.7282
Epoch 7/50
118/118 [==============================] - 27s 225ms/step - loss: 0.4229 - accuracy: 0.8299 - val_loss: 0.7404 - val_ac

###ResNet152V2 GS

In [ ]:
modelResnet152V2_gs = create_resnetv2_model(152,X_train_gs.shape[1:])

In [ ]:
history_modelResnet152V2_gs = modelResnet152V2_gs.fit(X_train_gs, y_train_gs, batch_size=batch_size, epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 41s 241ms/step - loss: 1.4671 - accuracy: 0.4858 - val_loss: 1.8781 - val_accuracy: 0.2763
Epoch 2/50
118/118 [==============================] - 26s 222ms/step - loss: 0.7750 - accuracy: 0.6946 - val_loss: 2.5507 - val_accuracy: 0.3099
Epoch 3/50
118/118 [==============================] - 26s 223ms/step - loss: 0.6520 - accuracy: 0.7615 - val_loss: 3.8708 - val_accuracy: 0.1679
Epoch 4/50
118/118 [==============================] - 26s 224ms/step - loss: 0.5199 - accuracy: 0.8008 - val_loss: 1.6486 - val_accuracy: 0.3298
Epoch 5/50
118/118 [==============================] - 26s 224ms/step - loss: 0.4509 - accuracy: 0.8372 - val_loss: 11.7862 - val_accuracy: 0.3145
Epoch 6/50
118/118 [==============================] - 26s 224ms/step - loss: 0.4711 - accuracy: 0.8237 - val_loss: 0.9531 - val_accuracy: 0.7023
Epoch 7/50
118/118 [==============================] - 26s 225ms/step - loss: 0.3147 - accuracy: 0.8879 - val_loss: 2.3541 - val_a

###ResNet152V2 RGB With TFL

In [ ]:
modelResnet152V2_tfl = create_resnetv2_model(152,X_train_rgb.shape[1:],True)

234553344/234545216 [==============================] - 3s 0us/step


In [ ]:
history_modelResnet152V2_tfl = modelResnet152V2_tfl.fit(X_train_rgb, y_train_rgb, batch_size=batch_size, epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 40s 238ms/step - loss: 18.3001 - accuracy: 0.5669 - val_loss: 0.9730 - val_accuracy: 0.8336
Epoch 2/50
118/118 [==============================] - 25s 210ms/step - loss: 0.5376 - accuracy: 0.8583 - val_loss: 0.4679 - val_accuracy: 0.8565
Epoch 3/50
118/118 [==============================] - 25s 211ms/step - loss: 0.2618 - accuracy: 0.9231 - val_loss: 0.5098 - val_accuracy: 0.8809
Epoch 4/50
118/118 [==============================] - 25s 211ms/step - loss: 0.1992 - accuracy: 0.9369 - val_loss: 0.4700 - val_accuracy: 0.8855
Epoch 5/50
118/118 [==============================] - 25s 211ms/step - loss: 0.1124 - accuracy: 0.9594 - val_loss: 0.6277 - val_accuracy: 0.8855
Epoch 6/50
118/118 [==============================] - 25s 211ms/step - loss: 0.1165 - accuracy: 0.9630 - val_loss: 0.6767 - val_accuracy: 0.8626
Epoch 7/50
118/118 [==============================] - 25s 211ms/step - loss: 0.0944 - accuracy: 0.9624 - val_loss: 0.5334 - val_a

#Training With Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=[0.5,1.0],
    width_shift_range=0.15,  
    height_shift_range=0.15,  
    horizontal_flip=True
) 

In [ ]:
X_train_gs_da = X_train_gs.copy()
X_train_rgb_da = X_train_rgb.copy()
datagen.fit(X_train_gs_da)
datagen.fit(X_train_rgb_da)

##Training Custom Model

###Custom Model RGB

In [ ]:
modelCNN_rgb_da = create_custom_models(X_train_rgb.shape[1:])

In [ ]:
history_modelCNN_rgb_da = modelCNN_rgb_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 11s 85ms/step - loss: 1.3769 - accuracy: 0.3224 - val_loss: 1.3251 - val_accuracy: 0.3588
Epoch 2/50
118/118 [==============================] - 10s 82ms/step - loss: 1.2856 - accuracy: 0.3596 - val_loss: 1.3226 - val_accuracy: 0.2992
Epoch 3/50
118/118 [==============================] - 10s 82ms/step - loss: 1.2472 - accuracy: 0.4116 - val_loss: 1.2398 - val_accuracy: 0.4855
Epoch 4/50
118/118 [==============================] - 10s 82ms/step - loss: 1.1825 - accuracy: 0.4673 - val_loss: 1.2704 - val_accuracy: 0.3832
Epoch 5/50
118/118 [==============================] - 10s 82ms/step - loss: 1.1944 - accuracy: 0.4753 - val_loss: 1.2693 - val_accuracy: 0.3893
Epoch 6/50
118/118 [==============================] - 11s 94ms/step - loss: 1.1673 - accuracy: 0.4776 - val_loss: 1.1315 - val_accuracy: 0.4321
Epoch 7/50
118/118 [==============================] - 10s 83ms/step - loss: 1.1299 - accuracy: 0.4884 - val_loss: 1.0913 - val_accuracy:

###Custom Model GS

In [ ]:
modelCNN_gs_da = create_custom_models(X_train_gs.shape[1:])

In [ ]:
history_modelCNN_gs_da = modelCNN_gs_da.fit(datagen.flow(X_train_gs_da, y_train_gs, batch_size=batch_size), epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 6s 42ms/step - loss: 1.3681 - accuracy: 0.2872 - val_loss: 1.3036 - val_accuracy: 0.3023
Epoch 2/50
118/118 [==============================] - 5s 39ms/step - loss: 1.2579 - accuracy: 0.3926 - val_loss: 1.2194 - val_accuracy: 0.3267
Epoch 3/50
118/118 [==============================] - 5s 39ms/step - loss: 1.2463 - accuracy: 0.3978 - val_loss: 1.2025 - val_accuracy: 0.3908
Epoch 4/50
118/118 [==============================] - 5s 39ms/step - loss: 1.2056 - accuracy: 0.4489 - val_loss: 1.2496 - val_accuracy: 0.3450
Epoch 5/50
118/118 [==============================] - 5s 39ms/step - loss: 1.1589 - accuracy: 0.4712 - val_loss: 1.2236 - val_accuracy: 0.3863
Epoch 6/50
118/118 [==============================] - 5s 39ms/step - loss: 1.1595 - accuracy: 0.4695 - val_loss: 1.0786 - val_accuracy: 0.5374
Epoch 7/50
118/118 [==============================] - 5s 39ms/step - loss: 1.1768 - accuracy: 0.4576 - val_loss: 1.1039 - val_accuracy: 0.5084

##Training VGG16

###VGG16 RGB

In [ ]:
modelVGG16_rgb_da = create_vgg_model(16,X_train_rgb.shape[1:])

In [ ]:
history_modelVGG16_rgb_da = modelVGG16_rgb_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 59s 191ms/step - loss: 1.3634 - accuracy: 0.3043 - val_loss: 1.3867 - val_accuracy: 0.2672
Epoch 2/50
118/118 [==============================] - 15s 123ms/step - loss: 1.3465 - accuracy: 0.3161 - val_loss: 1.3777 - val_accuracy: 0.2687
Epoch 3/50
118/118 [==============================] - 14s 122ms/step - loss: 1.3583 - accuracy: 0.2963 - val_loss: 1.3860 - val_accuracy: 0.2763
Epoch 4/50
118/118 [==============================] - 14s 122ms/step - loss: 1.3503 - accuracy: 0.2927 - val_loss: 1.3823 - val_accuracy: 0.2687
Epoch 5/50
118/118 [==============================] - 14s 123ms/step - loss: 1.3549 - accuracy: 0.2918 - val_loss: 1.3832 - val_accuracy: 0.2687
Epoch 6/50
118/118 [==============================] - 15s 123ms/step - loss: 1.3488 - accuracy: 0.2988 - val_loss: 1.3847 - val_accuracy: 0.2687
Epoch 7/50
118/118 [==============================] - 15s 123ms/step - loss: 1.3593 - accuracy: 0.2733 - val_loss: 1.3891 - val_ac

###VGG16 GS

In [ ]:
modelVGG16_gs_da = create_vgg_model(16,X_train_gs.shape[1:])

In [ ]:
history_modelVGG16_gs_da = modelVGG16_gs_da.fit(datagen.flow(X_train_gs_da, y_train_gs, batch_size=batch_size), epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 16s 128ms/step - loss: 1.3616 - accuracy: 0.2739 - val_loss: 1.3861 - val_accuracy: 0.2687
Epoch 2/50
118/118 [==============================] - 15s 123ms/step - loss: 1.3543 - accuracy: 0.2716 - val_loss: 1.3763 - val_accuracy: 0.2763
Epoch 3/50
118/118 [==============================] - 15s 123ms/step - loss: 1.3587 - accuracy: 0.2777 - val_loss: 1.3908 - val_accuracy: 0.2687
Epoch 4/50
118/118 [==============================] - 15s 123ms/step - loss: 1.3443 - accuracy: 0.2717 - val_loss: 1.3842 - val_accuracy: 0.2672
Epoch 5/50
118/118 [==============================] - 15s 123ms/step - loss: 1.3555 - accuracy: 0.2916 - val_loss: 1.3842 - val_accuracy: 0.2672
Epoch 6/50
118/118 [==============================] - 15s 123ms/step - loss: 1.3602 - accuracy: 0.2883 - val_loss: 1.3855 - val_accuracy: 0.2687
Epoch 7/50
118/118 [==============================] - 14s 123ms/step - loss: 1.3520 - accuracy: 0.2865 - val_loss: 1.3835 - val_ac

###VGG16 RGB With TFL

In [ ]:
modelVGG16_tfl_da = create_vgg_model(16,X_train_rgb.shape[1:],True)

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
history_modelVGG16_tfl_da = modelVGG16_tfl_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 11s 86ms/step - loss: 2.6388 - accuracy: 0.4566 - val_loss: 1.0025 - val_accuracy: 0.5298
Epoch 2/50
118/118 [==============================] - 10s 85ms/step - loss: 0.8795 - accuracy: 0.6336 - val_loss: 0.6881 - val_accuracy: 0.7313
Epoch 3/50
118/118 [==============================] - 10s 84ms/step - loss: 0.7329 - accuracy: 0.7124 - val_loss: 0.6676 - val_accuracy: 0.7344
Epoch 4/50
118/118 [==============================] - 10s 84ms/step - loss: 0.8096 - accuracy: 0.6668 - val_loss: 0.7659 - val_accuracy: 0.6611
Epoch 5/50
118/118 [==============================] - 10s 83ms/step - loss: 0.6770 - accuracy: 0.7230 - val_loss: 0.8316 - val_accuracy: 0.6458
Epoch 6/50
118/118 [==============================] - 10s 83ms/step - loss: 0.6856 - accuracy: 0.7267 - val_loss: 0.6937 - val_accuracy: 0.7130
Epoch 7/50
118/118 [==============================] - 10s 83ms/step - loss: 0.6998 - accuracy: 0.7199 - val_loss: 0.5573 - val_accuracy:

##Training VGG19

###VGG19 RGB

In [ ]:
modelVGG19_rgb_da = create_vgg_model(19,X_train_rgb.shape[1:])

In [ ]:
history_modelVGG19_rgb_da = modelVGG19_rgb_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 18s 148ms/step - loss: 1.3605 - accuracy: 0.2921 - val_loss: 1.3827 - val_accuracy: 0.2672
Epoch 2/50
118/118 [==============================] - 17s 144ms/step - loss: 1.3497 - accuracy: 0.2920 - val_loss: 1.3824 - val_accuracy: 0.2672
Epoch 3/50
118/118 [==============================] - 17s 144ms/step - loss: 1.3552 - accuracy: 0.2924 - val_loss: 1.3864 - val_accuracy: 0.2687
Epoch 4/50
118/118 [==============================] - 17s 144ms/step - loss: 1.3422 - accuracy: 0.3021 - val_loss: 1.3806 - val_accuracy: 0.2687
Epoch 5/50
118/118 [==============================] - 17s 144ms/step - loss: 1.3587 - accuracy: 0.2979 - val_loss: 1.3970 - val_accuracy: 0.2672
Epoch 6/50
118/118 [==============================] - 17s 144ms/step - loss: 1.3529 - accuracy: 0.2785 - val_loss: 1.3917 - val_accuracy: 0.2672
Epoch 7/50
118/118 [==============================] - 17s 144ms/step - loss: 1.3644 - accuracy: 0.2854 - val_loss: 1.3958 - val_ac

###VGG19 GS

In [ ]:
modelVGG19_gs_da = create_vgg_model(19,X_train_gs.shape[1:])

In [ ]:
history_modelVGG19_gs_da = modelVGG19_gs_da.fit(datagen.flow(X_train_gs_da, y_train_gs, batch_size=batch_size), epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 19s 149ms/step - loss: 1.3657 - accuracy: 0.3012 - val_loss: 1.3784 - val_accuracy: 0.2687
Epoch 2/50
118/118 [==============================] - 17s 145ms/step - loss: 1.3491 - accuracy: 0.2898 - val_loss: 1.3776 - val_accuracy: 0.2687
Epoch 3/50
118/118 [==============================] - 17s 146ms/step - loss: 1.3539 - accuracy: 0.2766 - val_loss: 1.3846 - val_accuracy: 0.2687
Epoch 4/50
118/118 [==============================] - 17s 146ms/step - loss: 1.3569 - accuracy: 0.3082 - val_loss: 1.3892 - val_accuracy: 0.2672
Epoch 5/50
118/118 [==============================] - 17s 146ms/step - loss: 1.3530 - accuracy: 0.2840 - val_loss: 1.3827 - val_accuracy: 0.2672
Epoch 6/50
118/118 [==============================] - 17s 146ms/step - loss: 1.3563 - accuracy: 0.2934 - val_loss: 1.3824 - val_accuracy: 0.2687
Epoch 7/50
118/118 [==============================] - 17s 145ms/step - loss: 1.3451 - accuracy: 0.2923 - val_loss: 1.3809 - val_ac

###VGG19 RGB With TFL

In [ ]:
modelVGG19_tfl_da = create_vgg_model(19,X_train_rgb.shape[1:], True)

80142336/80134624 [==============================] - 1s 0us/step


In [ ]:
history_modelVGG19_tfl_da = modelVGG19_tfl_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 22s 150ms/step - loss: 3.3220 - accuracy: 0.4238 - val_loss: 0.8634 - val_accuracy: 0.6458
Epoch 2/50
118/118 [==============================] - 11s 90ms/step - loss: 1.0200 - accuracy: 0.5525 - val_loss: 1.0194 - val_accuracy: 0.5206
Epoch 3/50
118/118 [==============================] - 10s 88ms/step - loss: 0.9093 - accuracy: 0.6178 - val_loss: 0.8390 - val_accuracy: 0.6290
Epoch 4/50
118/118 [==============================] - 10s 88ms/step - loss: 0.8696 - accuracy: 0.6353 - val_loss: 0.7293 - val_accuracy: 0.6977
Epoch 5/50
118/118 [==============================] - 10s 87ms/step - loss: 0.7977 - accuracy: 0.6629 - val_loss: 0.6850 - val_accuracy: 0.7344
Epoch 6/50
118/118 [==============================] - 10s 87ms/step - loss: 0.7995 - accuracy: 0.6531 - val_loss: 0.6578 - val_accuracy: 0.7420
Epoch 7/50
118/118 [==============================] - 10s 87ms/step - loss: 0.8311 - accuracy: 0.6466 - val_loss: 0.7255 - val_accuracy

##Training Xception

###Xception RGB

In [ ]:
modelXception_rgb_da = create_Xception_model(X_train_rgb.shape[1:])

In [ ]:
history_modelXception_rgb_da = modelXception_rgb_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 41s 308ms/step - loss: 1.4293 - accuracy: 0.4090 - val_loss: 1.3853 - val_accuracy: 0.2672
Epoch 2/50
118/118 [==============================] - 36s 302ms/step - loss: 1.1454 - accuracy: 0.5047 - val_loss: 1.3835 - val_accuracy: 0.2763
Epoch 3/50
118/118 [==============================] - 36s 302ms/step - loss: 0.9365 - accuracy: 0.6201 - val_loss: 1.3876 - val_accuracy: 0.2809
Epoch 4/50
118/118 [==============================] - 36s 301ms/step - loss: 0.8568 - accuracy: 0.6535 - val_loss: 1.3021 - val_accuracy: 0.4519
Epoch 5/50
118/118 [==============================] - 36s 301ms/step - loss: 0.7692 - accuracy: 0.7092 - val_loss: 1.2610 - val_accuracy: 0.4947
Epoch 6/50
118/118 [==============================] - 36s 301ms/step - loss: 0.6724 - accuracy: 0.7509 - val_loss: 1.0114 - val_accuracy: 0.5618
Epoch 7/50
118/118 [==============================] - 35s 300ms/step - loss: 0.5732 - accuracy: 0.7970 - val_loss: 1.0228 - val_ac

###Xception GS

In [ ]:
modelXception_gs_da = create_Xception_model(X_train_gs.shape[1:])

In [ ]:
history_modelXception_gs_da = modelXception_gs_da.fit(datagen.flow(X_train_gs_da, y_train_gs, batch_size=batch_size), epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 41s 307ms/step - loss: 1.4042 - accuracy: 0.4028 - val_loss: 1.3853 - val_accuracy: 0.2763
Epoch 2/50
118/118 [==============================] - 35s 298ms/step - loss: 1.0806 - accuracy: 0.5482 - val_loss: 1.3879 - val_accuracy: 0.2763
Epoch 3/50
118/118 [==============================] - 35s 299ms/step - loss: 0.9339 - accuracy: 0.6243 - val_loss: 1.3970 - val_accuracy: 0.1878
Epoch 4/50
118/118 [==============================] - 35s 298ms/step - loss: 0.7961 - accuracy: 0.6819 - val_loss: 1.3896 - val_accuracy: 0.3206
Epoch 5/50
118/118 [==============================] - 35s 299ms/step - loss: 0.7667 - accuracy: 0.6975 - val_loss: 2.6934 - val_accuracy: 0.3649
Epoch 6/50
118/118 [==============================] - 35s 298ms/step - loss: 0.6573 - accuracy: 0.7595 - val_loss: 0.9074 - val_accuracy: 0.6092
Epoch 7/50
118/118 [==============================] - 35s 298ms/step - loss: 0.5703 - accuracy: 0.7916 - val_loss: 1.1463 - val_ac

###Xception RGB With TFL

In [ ]:
modelXception_tfl_da = create_Xception_model(X_train_rgb.shape[1:], True)

In [ ]:
history_modelXception_tfl_da = modelXception_tfl_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 34s 115ms/step - loss: 10.8791 - accuracy: 0.4999 - val_loss: 0.6755 - val_accuracy: 0.7511
Epoch 2/50
118/118 [==============================] - 11s 93ms/step - loss: 0.8409 - accuracy: 0.6802 - val_loss: 0.8987 - val_accuracy: 0.6519
Epoch 3/50
118/118 [==============================] - 11s 89ms/step - loss: 0.8063 - accuracy: 0.6957 - val_loss: 0.6316 - val_accuracy: 0.7603
Epoch 4/50
118/118 [==============================] - 10s 88ms/step - loss: 0.7524 - accuracy: 0.7119 - val_loss: 0.5742 - val_accuracy: 0.7557
Epoch 5/50
118/118 [==============================] - 11s 89ms/step - loss: 0.7584 - accuracy: 0.7027 - val_loss: 0.5924 - val_accuracy: 0.7740
Epoch 6/50
118/118 [==============================] - 10s 88ms/step - loss: 0.6831 - accuracy: 0.7383 - val_loss: 0.5724 - val_accuracy: 0.7695
Epoch 7/50
118/118 [==============================] - 11s 94ms/step - loss: 0.6581 - accuracy: 0.7410 - val_loss: 0.5589 - val_accurac

##Training ResNet

###ResNet50 RGB

In [ ]:
modelResnet50_rgb_da = create_resnet_model(50,X_train_rgb.shape[1:])

In [ ]:
history_modelResnet50_rgb_da = modelResnet50_rgb_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 23s 149ms/step - loss: 2.5358 - accuracy: 0.3860 - val_loss: 1.3782 - val_accuracy: 0.2733
Epoch 2/50
118/118 [==============================] - 16s 135ms/step - loss: 1.2455 - accuracy: 0.4951 - val_loss: 2.4602 - val_accuracy: 0.2702
Epoch 3/50
118/118 [==============================] - 16s 134ms/step - loss: 1.1830 - accuracy: 0.5768 - val_loss: 1.4084 - val_accuracy: 0.2794
Epoch 4/50
118/118 [==============================] - 16s 134ms/step - loss: 0.9573 - accuracy: 0.6142 - val_loss: 2.2228 - val_accuracy: 0.1405
Epoch 5/50
118/118 [==============================] - 16s 134ms/step - loss: 0.8904 - accuracy: 0.6403 - val_loss: 1.7523 - val_accuracy: 0.3939
Epoch 6/50
118/118 [==============================] - 16s 133ms/step - loss: 0.8169 - accuracy: 0.6728 - val_loss: 1.2859 - val_accuracy: 0.4412
Epoch 7/50
118/118 [==============================] - 16s 134ms/step - loss: 0.8122 - accuracy: 0.6809 - val_loss: 3.1799 - val_ac

###ResNet50 GS

In [ ]:
modelResnet50_gs_da = create_resnet_model(50,X_train_gs.shape[1:])

In [ ]:
history_modelResnet50_gs_da = modelResnet50_gs_da.fit(datagen.flow(X_train_gs_da, y_train_gs, batch_size=batch_size), epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 21s 140ms/step - loss: 2.5068 - accuracy: 0.3903 - val_loss: 2.9373 - val_accuracy: 0.2763
Epoch 2/50
118/118 [==============================] - 16s 132ms/step - loss: 1.1979 - accuracy: 0.5428 - val_loss: 2.6244 - val_accuracy: 0.2794
Epoch 3/50
118/118 [==============================] - 16s 131ms/step - loss: 1.1994 - accuracy: 0.5826 - val_loss: 1.5853 - val_accuracy: 0.1420
Epoch 4/50
118/118 [==============================] - 16s 131ms/step - loss: 0.9588 - accuracy: 0.6274 - val_loss: 4.5140 - val_accuracy: 0.2687
Epoch 5/50
118/118 [==============================] - 16s 132ms/step - loss: 0.9192 - accuracy: 0.6498 - val_loss: 2.6736 - val_accuracy: 0.3511
Epoch 6/50
118/118 [==============================] - 16s 132ms/step - loss: 0.8965 - accuracy: 0.6479 - val_loss: 1.5046 - val_accuracy: 0.4534
Epoch 7/50
118/118 [==============================] - 16s 131ms/step - loss: 0.8463 - accuracy: 0.6751 - val_loss: 1.1823 - val_ac

###ResNet50 RGB With TFL

In [ ]:
modelResnet50_tfl_da = create_resnet_model(50,X_train_rgb.shape[1:],True)

94773248/94765736 [==============================] - 2s 0us/step


In [ ]:
history_modelResnet50_tfl_da = modelResnet50_tfl_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 51s 154ms/step - loss: 3.3972 - accuracy: 0.2929 - val_loss: 1.4619 - val_accuracy: 0.2992
Epoch 2/50
118/118 [==============================] - 16s 131ms/step - loss: 1.3382 - accuracy: 0.3523 - val_loss: 1.3388 - val_accuracy: 0.3328
Epoch 3/50
118/118 [==============================] - 16s 132ms/step - loss: 1.3147 - accuracy: 0.3496 - val_loss: 1.3531 - val_accuracy: 0.3817
Epoch 4/50
118/118 [==============================] - 15s 131ms/step - loss: 1.2845 - accuracy: 0.3647 - val_loss: 1.3677 - val_accuracy: 0.3771
Epoch 5/50
118/118 [==============================] - 16s 131ms/step - loss: 1.2924 - accuracy: 0.3666 - val_loss: 1.2991 - val_accuracy: 0.3756
Epoch 6/50
118/118 [==============================] - 16s 131ms/step - loss: 1.2847 - accuracy: 0.3741 - val_loss: 1.2314 - val_accuracy: 0.4382
Epoch 7/50
118/118 [==============================] - 16s 131ms/step - loss: 1.2788 - accuracy: 0.3696 - val_loss: 1.2620 - val_ac

###ResNet101 RGB

In [ ]:
modelResnet101_rgb_da = create_resnet_model(101,X_train_rgb.shape[1:])

In [ ]:
history_modelResnet101_rgb_da = modelResnet101_rgb_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 33s 194ms/step - loss: 2.3102 - accuracy: 0.3441 - val_loss: 2.9435 - val_accuracy: 0.2763
Epoch 2/50
118/118 [==============================] - 20s 169ms/step - loss: 1.1763 - accuracy: 0.5250 - val_loss: 1.8352 - val_accuracy: 0.3511
Epoch 3/50
118/118 [==============================] - 20s 169ms/step - loss: 1.1145 - accuracy: 0.5546 - val_loss: 1.6789 - val_accuracy: 0.3573
Epoch 4/50
118/118 [==============================] - 20s 167ms/step - loss: 1.0826 - accuracy: 0.5280 - val_loss: 301.0989 - val_accuracy: 0.3053
Epoch 5/50
118/118 [==============================] - 20s 166ms/step - loss: 1.1304 - accuracy: 0.5120 - val_loss: 1.5056 - val_accuracy: 0.4580
Epoch 6/50
118/118 [==============================] - 20s 168ms/step - loss: 1.0549 - accuracy: 0.5572 - val_loss: 0.9810 - val_accuracy: 0.5817
Epoch 7/50
118/118 [==============================] - 20s 168ms/step - loss: 0.9752 - accuracy: 0.5969 - val_loss: 1.6002 - val_

###ResNet101 GS

In [ ]:
modelResnet101_gs_da = create_resnet_model(101,X_train_gs.shape[1:])

In [ ]:
history_modelResnet101_gs_da = modelResnet101_gs_da.fit(datagen.flow(X_train_gs_da, y_train_gs, batch_size=batch_size), epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 30s 180ms/step - loss: 2.4597 - accuracy: 0.3900 - val_loss: 2.6078 - val_accuracy: 0.2763
Epoch 2/50
118/118 [==============================] - 20s 167ms/step - loss: 1.1697 - accuracy: 0.5454 - val_loss: 1.7395 - val_accuracy: 0.3863
Epoch 3/50
118/118 [==============================] - 19s 165ms/step - loss: 1.1194 - accuracy: 0.5307 - val_loss: 2.1464 - val_accuracy: 0.3053
Epoch 4/50
118/118 [==============================] - 19s 163ms/step - loss: 0.9507 - accuracy: 0.6272 - val_loss: 2.9461 - val_accuracy: 0.3069
Epoch 5/50
118/118 [==============================] - 19s 164ms/step - loss: 0.9849 - accuracy: 0.6112 - val_loss: 12.1090 - val_accuracy: 0.1740
Epoch 6/50
118/118 [==============================] - 19s 165ms/step - loss: 0.9035 - accuracy: 0.6454 - val_loss: 33.0573 - val_accuracy: 0.1863
Epoch 7/50
118/118 [==============================] - 19s 165ms/step - loss: 0.9360 - accuracy: 0.6132 - val_loss: 1.3840 - val_

###ResNet101 RGB With TFL

In [ ]:
modelResnet101_tfl_da = create_resnet_model(101,X_train_rgb.shape[1:], True)

171450368/171446536 [==============================] - 2s 0us/step


In [ ]:
history_modelResnet101_tfl_da = modelResnet101_tfl_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 29s 186ms/step - loss: 2.4525 - accuracy: 0.2808 - val_loss: 1.2403 - val_accuracy: 0.4168
Epoch 2/50
118/118 [==============================] - 20s 167ms/step - loss: 1.3133 - accuracy: 0.3724 - val_loss: 1.3272 - val_accuracy: 0.4031
Epoch 3/50
118/118 [==============================] - 20s 168ms/step - loss: 1.2724 - accuracy: 0.4081 - val_loss: 1.2052 - val_accuracy: 0.4351
Epoch 4/50
118/118 [==============================] - 20s 168ms/step - loss: 1.2778 - accuracy: 0.3800 - val_loss: 1.2789 - val_accuracy: 0.4336
Epoch 5/50
118/118 [==============================] - 20s 168ms/step - loss: 1.2467 - accuracy: 0.4135 - val_loss: 1.2250 - val_accuracy: 0.4092
Epoch 6/50
118/118 [==============================] - 20s 167ms/step - loss: 1.2448 - accuracy: 0.4224 - val_loss: 1.1647 - val_accuracy: 0.4580
Epoch 7/50
118/118 [==============================] - 20s 168ms/step - loss: 1.2372 - accuracy: 0.4226 - val_loss: 1.1504 - val_ac

###ResNet152 RGB

In [ ]:
modelResnet152_rgb_da = create_resnet_model(152,X_train_rgb.shape[1:])

In [ ]:
history_modelResnet152_rgb_da = modelResnet152_rgb_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 46s 268ms/step - loss: 2.2973 - accuracy: 0.3361 - val_loss: 2.0277 - val_accuracy: 0.2763
Epoch 2/50
118/118 [==============================] - 28s 233ms/step - loss: 1.2714 - accuracy: 0.4872 - val_loss: 2.0143 - val_accuracy: 0.2855
Epoch 3/50
118/118 [==============================] - 28s 234ms/step - loss: 1.1538 - accuracy: 0.5064 - val_loss: 1.4049 - val_accuracy: 0.3664
Epoch 4/50
118/118 [==============================] - 28s 235ms/step - loss: 1.0331 - accuracy: 0.5630 - val_loss: 1.3779 - val_accuracy: 0.3802
Epoch 5/50
118/118 [==============================] - 28s 234ms/step - loss: 1.0338 - accuracy: 0.5717 - val_loss: 1.3216 - val_accuracy: 0.4458
Epoch 6/50
118/118 [==============================] - 28s 234ms/step - loss: 0.9197 - accuracy: 0.6266 - val_loss: 606.7302 - val_accuracy: 0.3053
Epoch 7/50
118/118 [==============================] - 28s 234ms/step - loss: 1.1799 - accuracy: 0.5066 - val_loss: 3135.0681 - v

###ResNet152 GS

In [ ]:
modelResnet152_gs_da = create_resnet_model(152,X_train_gs.shape[1:])

In [ ]:
history_modelResnet152_gs_da = modelResnet152_gs_da.fit(datagen.flow(X_train_gs_da, y_train_gs, batch_size=batch_size), epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 44s 253ms/step - loss: 2.4278 - accuracy: 0.3462 - val_loss: 2.1041 - val_accuracy: 0.2687
Epoch 2/50
118/118 [==============================] - 28s 237ms/step - loss: 1.3352 - accuracy: 0.3963 - val_loss: 2.1571 - val_accuracy: 0.2763
Epoch 3/50
118/118 [==============================] - 28s 235ms/step - loss: 1.2092 - accuracy: 0.4909 - val_loss: 2.0922 - val_accuracy: 0.3710
Epoch 4/50
118/118 [==============================] - 28s 233ms/step - loss: 1.0150 - accuracy: 0.6121 - val_loss: 2.6425 - val_accuracy: 0.2977
Epoch 5/50
118/118 [==============================] - 28s 236ms/step - loss: 1.0028 - accuracy: 0.5800 - val_loss: 1.6189 - val_accuracy: 0.2565
Epoch 6/50
118/118 [==============================] - 28s 235ms/step - loss: 0.9541 - accuracy: 0.6144 - val_loss: 2.1211 - val_accuracy: 0.4107
Epoch 7/50
118/118 [==============================] - 28s 234ms/step - loss: 0.8763 - accuracy: 0.6525 - val_loss: 44.0234 - val_a

###ResNet152 RGB With TFL

In [ ]:
modelResnet152_tfl_da = create_resnet_model(152,X_train_rgb.shape[1:], True)

234700800/234698864 [==============================] - 6s 0us/step


In [ ]:
history_modelResnet152_tfl_da = modelResnet152_tfl_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 40s 255ms/step - loss: 2.0872 - accuracy: 0.3167 - val_loss: 1.4230 - val_accuracy: 0.3053
Epoch 2/50
118/118 [==============================] - 27s 229ms/step - loss: 1.3419 - accuracy: 0.3574 - val_loss: 1.3051 - val_accuracy: 0.3344
Epoch 3/50
118/118 [==============================] - 27s 229ms/step - loss: 1.2952 - accuracy: 0.3748 - val_loss: 1.2711 - val_accuracy: 0.3740
Epoch 4/50
118/118 [==============================] - 27s 228ms/step - loss: 1.2698 - accuracy: 0.4048 - val_loss: 1.2450 - val_accuracy: 0.4351
Epoch 5/50
118/118 [==============================] - 27s 228ms/step - loss: 1.2653 - accuracy: 0.3853 - val_loss: 1.2283 - val_accuracy: 0.3679
Epoch 6/50
118/118 [==============================] - 27s 228ms/step - loss: 1.2775 - accuracy: 0.3940 - val_loss: 1.2219 - val_accuracy: 0.4137
Epoch 7/50
118/118 [==============================] - 27s 229ms/step - loss: 1.2482 - accuracy: 0.4197 - val_loss: 1.2109 - val_ac

##Training ResNetV2

###ResNet50V2 RGB

In [ ]:
modelResnet50V2_rgb_da = create_resnetv2_model(50,X_train_rgb.shape[1:])

In [ ]:
history_modelResnet50V2_rgb_da = modelResnet50V2_rgb_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 63s 230ms/step - loss: 1.4075 - accuracy: 0.5137 - val_loss: 1.1372 - val_accuracy: 0.5466
Epoch 2/50
118/118 [==============================] - 24s 199ms/step - loss: 0.8214 - accuracy: 0.6706 - val_loss: 2.2467 - val_accuracy: 0.5130
Epoch 3/50
118/118 [==============================] - 24s 200ms/step - loss: 0.6442 - accuracy: 0.7418 - val_loss: 1.5670 - val_accuracy: 0.4580
Epoch 4/50
118/118 [==============================] - 24s 201ms/step - loss: 0.5117 - accuracy: 0.7990 - val_loss: 1.0139 - val_accuracy: 0.6015
Epoch 5/50
118/118 [==============================] - 24s 201ms/step - loss: 0.4529 - accuracy: 0.8253 - val_loss: 1.9453 - val_accuracy: 0.3710
Epoch 6/50
118/118 [==============================] - 24s 201ms/step - loss: 0.3092 - accuracy: 0.8784 - val_loss: 1.2074 - val_accuracy: 0.5878
Epoch 7/50
118/118 [==============================] - 24s 201ms/step - loss: 0.2732 - accuracy: 0.8965 - val_loss: 1.6221 - val_ac

###ResNet50V2 GS

In [ ]:
modelResnet50V2_gs_da = create_resnetv2_model(50,X_train_gs.shape[1:])

In [ ]:
history_modelResnet50V2_gs_da = modelResnet50V2_gs_da.fit(datagen.flow(X_train_gs_da, y_train_gs, batch_size=batch_size), epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 30s 207ms/step - loss: 1.3551 - accuracy: 0.5235 - val_loss: 3.2661 - val_accuracy: 0.1878
Epoch 2/50
118/118 [==============================] - 23s 197ms/step - loss: 0.6700 - accuracy: 0.7267 - val_loss: 3.1854 - val_accuracy: 0.2763
Epoch 3/50
118/118 [==============================] - 23s 197ms/step - loss: 0.5622 - accuracy: 0.7770 - val_loss: 2.0442 - val_accuracy: 0.3313
Epoch 4/50
118/118 [==============================] - 23s 198ms/step - loss: 0.4194 - accuracy: 0.8448 - val_loss: 2.5337 - val_accuracy: 0.5313
Epoch 5/50
118/118 [==============================] - 23s 198ms/step - loss: 0.2796 - accuracy: 0.9010 - val_loss: 4.2868 - val_accuracy: 0.6656
Epoch 6/50
118/118 [==============================] - 23s 199ms/step - loss: 0.3018 - accuracy: 0.8831 - val_loss: 1.2617 - val_accuracy: 0.7695
Epoch 7/50
118/118 [==============================] - 23s 199ms/step - loss: 0.2305 - accuracy: 0.9153 - val_loss: 7.5725 - val_ac

###ResNet50V2 RGB With TFL

In [ ]:
modelResnet50V2_tfl_da = create_resnetv2_model(50,X_train_rgb.shape[1:],True)

94674944/94668760 [==============================] - 3s 0us/step


In [ ]:
history_modelResnet50V2_tfl_da = modelResnet50V2_tfl_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 21s 143ms/step - loss: 17.3824 - accuracy: 0.5107 - val_loss: 0.7348 - val_accuracy: 0.7038
Epoch 2/50
118/118 [==============================] - 15s 125ms/step - loss: 0.8563 - accuracy: 0.6710 - val_loss: 0.7655 - val_accuracy: 0.7176
Epoch 3/50
118/118 [==============================] - 15s 125ms/step - loss: 0.7349 - accuracy: 0.7357 - val_loss: 0.5890 - val_accuracy: 0.7786
Epoch 4/50
118/118 [==============================] - 15s 124ms/step - loss: 0.7006 - accuracy: 0.7361 - val_loss: 0.6691 - val_accuracy: 0.7786
Epoch 5/50
118/118 [==============================] - 15s 124ms/step - loss: 0.7080 - accuracy: 0.7326 - val_loss: 0.6646 - val_accuracy: 0.7511
Epoch 6/50
118/118 [==============================] - 15s 125ms/step - loss: 0.6608 - accuracy: 0.7478 - val_loss: 0.5839 - val_accuracy: 0.7954
Epoch 7/50
118/118 [==============================] - 15s 125ms/step - loss: 0.6662 - accuracy: 0.7489 - val_loss: 0.6467 - val_a

###ResNet101V2 RGB

In [ ]:
modelResnet101V2_rgb_da = create_resnetv2_model(101,X_train_rgb.shape[1:])

In [ ]:
history_modelResnet101V2_rgb_da = modelResnet101V2_rgb_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 60s 386ms/step - loss: 1.5291 - accuracy: 0.4989 - val_loss: 2.6943 - val_accuracy: 0.2962
Epoch 2/50
118/118 [==============================] - 44s 372ms/step - loss: 0.7777 - accuracy: 0.6736 - val_loss: 3.0436 - val_accuracy: 0.1328
Epoch 3/50
118/118 [==============================] - 44s 372ms/step - loss: 0.6776 - accuracy: 0.7354 - val_loss: 33.4526 - val_accuracy: 0.6015
Epoch 4/50
118/118 [==============================] - 44s 372ms/step - loss: 0.5563 - accuracy: 0.7924 - val_loss: 17.6685 - val_accuracy: 0.4473
Epoch 5/50
118/118 [==============================] - 44s 372ms/step - loss: 0.5195 - accuracy: 0.7925 - val_loss: 1.7339 - val_accuracy: 0.5878
Epoch 6/50
118/118 [==============================] - 44s 372ms/step - loss: 0.3746 - accuracy: 0.8738 - val_loss: 1.3344 - val_accuracy: 0.6198
Epoch 7/50
118/118 [==============================] - 44s 372ms/step - loss: 0.3525 - accuracy: 0.8710 - val_loss: 0.4978 - val_

###ResNet101V2 GS

In [ ]:
modelResnet101V2_gs_da = create_resnetv2_model(101,X_train_gs.shape[1:])

In [ ]:
history_modelResnet101V2_gs_da = modelResnet101V2_gs_da.fit(datagen.flow(X_train_gs_da, y_train_gs, batch_size=batch_size), epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 56s 383ms/step - loss: 1.4959 - accuracy: 0.5133 - val_loss: 2.2630 - val_accuracy: 0.2321
Epoch 2/50
118/118 [==============================] - 43s 368ms/step - loss: 0.8110 - accuracy: 0.6604 - val_loss: 3.4206 - val_accuracy: 0.2809
Epoch 3/50
118/118 [==============================] - 44s 369ms/step - loss: 0.6291 - accuracy: 0.7555 - val_loss: 3.0920 - val_accuracy: 0.3084
Epoch 4/50
118/118 [==============================] - 44s 369ms/step - loss: 0.6081 - accuracy: 0.7777 - val_loss: 3.0204 - val_accuracy: 0.2351
Epoch 5/50
118/118 [==============================] - 44s 370ms/step - loss: 0.4975 - accuracy: 0.8160 - val_loss: 1.8256 - val_accuracy: 0.5328
Epoch 6/50
118/118 [==============================] - 43s 369ms/step - loss: 0.4267 - accuracy: 0.8416 - val_loss: 0.7910 - val_accuracy: 0.7466
Epoch 7/50
118/118 [==============================] - 44s 370ms/step - loss: 0.3386 - accuracy: 0.8701 - val_loss: 1.0491 - val_ac

###ResNet101V2 RGB With TFL

In [ ]:
modelResnet101V2_tfl_da = create_resnetv2_model(101,X_train_rgb.shape[1:],True)

171319296/171317808 [==============================] - 4s 0us/step


In [ ]:
history_modelResnet101V2_tfl_da = modelResnet101V2_tfl_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 27s 173ms/step - loss: 14.0491 - accuracy: 0.5299 - val_loss: 1.2159 - val_accuracy: 0.7450
Epoch 2/50
118/118 [==============================] - 19s 158ms/step - loss: 1.1684 - accuracy: 0.6797 - val_loss: 0.7564 - val_accuracy: 0.7481
Epoch 3/50
118/118 [==============================] - 19s 157ms/step - loss: 0.8266 - accuracy: 0.7056 - val_loss: 0.7382 - val_accuracy: 0.7359
Epoch 4/50
118/118 [==============================] - 19s 157ms/step - loss: 0.7627 - accuracy: 0.7228 - val_loss: 0.7433 - val_accuracy: 0.7237
Epoch 5/50
118/118 [==============================] - 19s 158ms/step - loss: 0.6642 - accuracy: 0.7434 - val_loss: 0.5790 - val_accuracy: 0.7863
Epoch 6/50
118/118 [==============================] - 19s 158ms/step - loss: 0.6826 - accuracy: 0.7441 - val_loss: 0.5845 - val_accuracy: 0.7786
Epoch 7/50
118/118 [==============================] - 19s 158ms/step - loss: 0.6797 - accuracy: 0.7529 - val_loss: 0.5234 - val_a

###ResNet152V2 RGB

In [ ]:
modelResnet152V2_rgb_da = create_resnetv2_model(152,X_train_rgb.shape[1:])

In [ ]:
history_modelResnet152V2_rgb_da = modelResnet152V2_rgb_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 89s 576ms/step - loss: 1.5569 - accuracy: 0.4206 - val_loss: 2.4559 - val_accuracy: 0.2855
Epoch 2/50
118/118 [==============================] - 66s 557ms/step - loss: 1.1836 - accuracy: 0.4782 - val_loss: 1.6725 - val_accuracy: 0.3313
Epoch 3/50
118/118 [==============================] - 66s 555ms/step - loss: 1.1221 - accuracy: 0.5366 - val_loss: 6.6184 - val_accuracy: 0.2275
Epoch 4/50
118/118 [==============================] - 66s 557ms/step - loss: 0.9994 - accuracy: 0.6006 - val_loss: 1.1883 - val_accuracy: 0.5328
Epoch 5/50
118/118 [==============================] - 66s 555ms/step - loss: 0.9974 - accuracy: 0.5973 - val_loss: 5.1733 - val_accuracy: 0.2641
Epoch 6/50
118/118 [==============================] - 66s 555ms/step - loss: 0.9823 - accuracy: 0.5958 - val_loss: 2.0053 - val_accuracy: 0.3893
Epoch 7/50
118/118 [==============================] - 66s 555ms/step - loss: 0.9407 - accuracy: 0.6109 - val_loss: 3.7693 - val_ac

###ResNet152V2 GS

In [ ]:
modelResnet152V2_gs_da = create_resnetv2_model(152,X_train_gs.shape[1:])

In [ ]:
history_modelResnet152V2_gs_da = modelResnet152V2_gs_da.fit(datagen.flow(X_train_gs_da, y_train_gs, batch_size=batch_size), epochs=50, validation_data=(X_val_gs, y_val_gs))

Epoch 1/50
118/118 [==============================] - 50s 262ms/step - loss: 1.6532 - accuracy: 0.3971 - val_loss: 1.6981 - val_accuracy: 0.2702
Epoch 2/50
118/118 [==============================] - 27s 228ms/step - loss: 1.1992 - accuracy: 0.4593 - val_loss: 1.4246 - val_accuracy: 0.3985
Epoch 3/50
118/118 [==============================] - 27s 230ms/step - loss: 1.1402 - accuracy: 0.5045 - val_loss: 1.5174 - val_accuracy: 0.3511
Epoch 4/50
118/118 [==============================] - 27s 233ms/step - loss: 1.0336 - accuracy: 0.5744 - val_loss: 1.7049 - val_accuracy: 0.3679
Epoch 5/50
118/118 [==============================] - 28s 235ms/step - loss: 0.9848 - accuracy: 0.5925 - val_loss: 1.4491 - val_accuracy: 0.4290
Epoch 6/50
118/118 [==============================] - 28s 236ms/step - loss: 0.8732 - accuracy: 0.6402 - val_loss: 3.0198 - val_accuracy: 0.3969
Epoch 7/50
118/118 [==============================] - 28s 235ms/step - loss: 0.9135 - accuracy: 0.6316 - val_loss: 2.6757 - val_ac

###ResNet152V2 RGB With TFL

In [ ]:
modelResnet152V2_tfl_da = create_resnetv2_model(152,X_train_rgb.shape[1:],True)

In [ ]:
history_modelResnet152V2_tfl_da = modelResnet152V2_tfl_da.fit(datagen.flow(X_train_rgb_da, y_train_rgb, batch_size=batch_size), epochs=50, validation_data=(X_val_rgb, y_val_rgb))

Epoch 1/50
118/118 [==============================] - 56s 142ms/step - loss: 14.8184 - accuracy: 0.5197 - val_loss: 0.9928 - val_accuracy: 0.7099
Epoch 2/50
118/118 [==============================] - 13s 107ms/step - loss: 0.9798 - accuracy: 0.6864 - val_loss: 0.6711 - val_accuracy: 0.7603
Epoch 3/50
118/118 [==============================] - 13s 108ms/step - loss: 0.8609 - accuracy: 0.6886 - val_loss: 0.6700 - val_accuracy: 0.7466
Epoch 4/50
118/118 [==============================] - 13s 107ms/step - loss: 0.7476 - accuracy: 0.7118 - val_loss: 0.7238 - val_accuracy: 0.7481
Epoch 5/50
118/118 [==============================] - 13s 108ms/step - loss: 0.7241 - accuracy: 0.7320 - val_loss: 0.5793 - val_accuracy: 0.7649
Epoch 6/50
118/118 [==============================] - 13s 108ms/step - loss: 0.6443 - accuracy: 0.7523 - val_loss: 0.6517 - val_accuracy: 0.7374
Epoch 7/50
118/118 [==============================] - 13s 108ms/step - loss: 0.6935 - accuracy: 0.7385 - val_loss: 0.8481 - val_a

#Save Models/Training

##Save Models Without Data Augmentation

In [ ]:
import pickle

In [ ]:
modelCNN_rgb.save('/content/drive/MyDrive/tfg/results/CNN/RGB/Normal/modelCNN_rgb.h')
modelCNN_rgb.save('/content/drive/MyDrive/tfg/results/CNN/RGB/Normal/modelCNN_rgb.h5')
with open('/content/drive/MyDrive/tfg/results/CNN/RGB/Normal/modelCNN_rgb', 'wb') as file_pi:
  pickle.dump(history_modelCNN_rgb.history, file_pi)

In [ ]:
modelCNN_gs.save('/content/drive/MyDrive/tfg/results/CNN/GS/Normal/modelCNN_gs.h')
modelCNN_gs.save('/content/drive/MyDrive/tfg/results/CNN/GS/Normal/modelCNN_gs.h5')
with open('/content/drive/MyDrive/tfg/results/CNN/GS/Normal/modelCNN_gs', 'wb') as file_pi:
  pickle.dump(history_modelCNN_gs.history, file_pi)

In [ ]:
modelVGG16_rgb.save('/content/drive/MyDrive/tfg/results/VGG16/RGB/Normal/modelVGG16_rgb.h')
modelVGG16_rgb.save('/content/drive/MyDrive/tfg/results/VGG16/RGB/Normal/modelVGG16_rgb.h5')
with open('/content/drive/MyDrive/tfg/results/VGG16/RGB/Normal/modelVGG16_rgb', 'wb') as file_pi:
  pickle.dump(history_modelVGG16_rgb.history, file_pi)

In [ ]:
modelVGG16_gs.save('/content/drive/MyDrive/tfg/results/VGG16/GS/Normal/modelVGG16_gs.h')
modelVGG16_gs.save('/content/drive/MyDrive/tfg/results/VGG16/GS/Normal/modelVGG16_gs.h5')
with open('/content/drive/MyDrive/tfg/results/VGG16/GS/Normal/modelVGG16_gs', 'wb') as file_pi:
  pickle.dump(history_modelVGG16_gs.history, file_pi)

In [ ]:
modelVGG16_tfl.save('/content/drive/MyDrive/tfg/results/VGG16/TFL/Normal/modelVGG16_tfl.h')
modelVGG16_tfl.save('/content/drive/MyDrive/tfg/results/VGG16/TFL/Normal/modelVGG16_tfl.h5')
with open('/content/drive/MyDrive/tfg/results/VGG16/TFL/Normal/modelVGG16_tfl', 'wb') as file_pi:
  pickle.dump(history_modelVGG16_tfl.history, file_pi)

In [ ]:
modelVGG19_rgb.save('/content/drive/MyDrive/tfg/results/VGG19/RGB/Normal/modelVGG19_rgb.h')
modelVGG19_rgb.save('/content/drive/MyDrive/tfg/results/VGG19/RGB/Normal/modelVGG19_rgb.h5')
with open('/content/drive/MyDrive/tfg/results/VGG19/RGB/Normal/modelVGG19_rgb', 'wb') as file_pi:
  pickle.dump(history_modelVGG19_rgb.history, file_pi)

In [ ]:
modelVGG19_gs.save('/content/drive/MyDrive/tfg/results/VGG19/GS/Normal/modelVGG19_gs.h')
modelVGG19_gs.save('/content/drive/MyDrive/tfg/results/VGG19/GS/Normal/modelVGG19_gs.h5')
with open('/content/drive/MyDrive/tfg/results/VGG19/GS/Normal/modelVGG19_gs', 'wb') as file_pi:
  pickle.dump(history_modelVGG19_gs.history, file_pi)

In [ ]:
modelVGG19_tfl.save('/content/drive/MyDrive/tfg/results/VGG19/TFL/Normal/modelVGG19_tfl.h')
modelVGG19_tfl.save('/content/drive/MyDrive/tfg/results/VGG19/TFL/Normal/modelVGG19_tfl.h5')
with open('/content/drive/MyDrive/tfg/results/VGG19/TFL/Normal/modelVGG19_tfl', 'wb') as file_pi:
  pickle.dump(history_modelVGG19_tfl.history, file_pi)

In [ ]:
modelXception_rgb.save('/content/drive/MyDrive/tfg/results/Xception/RGB/Normal/modelXception_rgb.h')
modelXception_rgb.save('/content/drive/MyDrive/tfg/results/Xception/RGB/Normal/modelXception_rgb.h5')
with open('/content/drive/MyDrive/tfg/results/Xception/RGB/Normal/modelXception_rgb', 'wb') as file_pi:
  pickle.dump(history_modelXception_rgb.history, file_pi)

In [ ]:
modelXception_tfl.save('/content/drive/MyDrive/tfg/results/Xception/TFL/Normal/modelXception_tfl.h')
modelXception_tfl.save('/content/drive/MyDrive/tfg/results/Xception/TFL/Normal/modelXception_tfl.h5')
with open('/content/drive/MyDrive/tfg/results/Xception/TFL/Normal/modelXception_tfl', 'wb') as file_pi:
  pickle.dump(history_modelXception_tfl.history, file_pi)

In [ ]:
modelXception_gs.save('/content/drive/MyDrive/tfg/results/Xception/GS/Normal/modelXception_gs.h')
modelXception_gs.save('/content/drive/MyDrive/tfg/results/Xception/GS/Normal/modelXception_gs.h5')
with open('/content/drive/MyDrive/tfg/results/Xception/GS/Normal/modelXception_gs', 'wb') as file_pi:
  pickle.dump(history_modelXception_gs.history, file_pi)

In [ ]:
modelResnet50_rgb.save('/content/drive/MyDrive/tfg/results/ResNet50/RGB/Normal/modelResnet50_rgb.h')
modelResnet50_rgb.save('/content/drive/MyDrive/tfg/results/ResNet50/RGB/Normal/modelResnet50_rgb.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet50/RGB/Normal/modelResnet50_rgb', 'wb') as file_pi:
  pickle.dump(history_modelResnet50_rgb.history, file_pi)

In [ ]:
modelResnet50_gs.save('/content/drive/MyDrive/tfg/results/ResNet50/GS/Normal/modelResnet50_gs.h')
modelResnet50_gs.save('/content/drive/MyDrive/tfg/results/ResNet50/GS/Normal/modelResnet50_gs.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet50/GS/Normal/modelResnet50_gs', 'wb') as file_pi:
  pickle.dump(history_modelResnet50_gs.history, file_pi)

In [ ]:
modelResnet50_tfl.save('/content/drive/MyDrive/tfg/results/ResNet50/TFL/Normal/modelResnet50_tfl.h')
modelResnet50_tfl.save('/content/drive/MyDrive/tfg/results/ResNet50/TFL/Normal/modelResnet50_tfl.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet50/TFL/Normal/modelResnet50_tfl', 'wb') as file_pi:
  pickle.dump(history_modelResnet50_tfl.history, file_pi)

In [ ]:
modelResnet101_rgb.save('/content/drive/MyDrive/tfg/results/ResNet101/RGB/Normal/modelResnet101_rgb.h')
modelResnet101_rgb.save('/content/drive/MyDrive/tfg/results/ResNet101/RGB/Normal/modelResnet101_rgb.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet101/RGB/Normal/modelResnet101_rgb', 'wb') as file_pi:
  pickle.dump(history_modelResnet101_rgb.history, file_pi)

In [ ]:
modelResnet101_gs.save('/content/drive/MyDrive/tfg/results/ResNet101/GS/Normal/modelResnet101_gs.h')
modelResnet101_gs.save('/content/drive/MyDrive/tfg/results/ResNet101/GS/Normal/modelResnet101_gs.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet101/GS/Normal/modelResnet101_gs', 'wb') as file_pi:
  pickle.dump(history_modelResnet101_gs.history, file_pi)

In [ ]:
modelResnet101_tfl.save('/content/drive/MyDrive/tfg/results/ResNet101/TFL/Normal/modelResnet101_tfl.h')
modelResnet101_tfl.save('/content/drive/MyDrive/tfg/results/ResNet101/TFL/Normal/modelResnet101_tfl.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet101/TFL/Normal/modelResnet101_tfl', 'wb') as file_pi:
  pickle.dump(history_modelResnet101_tfl.history, file_pi)

In [ ]:
modelResnet152_rgb.save('/content/drive/MyDrive/tfg/results/ResNet152/RGB/Normal/modelResnet152_rgb.h')
modelResnet152_rgb.save('/content/drive/MyDrive/tfg/results/ResNet152/RGB/Normal/modelResnet152_rgb.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet152/RGB/Normal/modelResnet152_rgb', 'wb') as file_pi:
  pickle.dump(history_modelResnet152_rgb.history, file_pi)

In [ ]:
modelResnet152_gs.save('/content/drive/MyDrive/tfg/results/ResNet152/GS/Normal/modelResnet152_gs.h')
modelResnet152_gs.save('/content/drive/MyDrive/tfg/results/ResNet152/GS/Normal/modelResnet152_gs.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet152/GS/Normal/modelResnet152_gs', 'wb') as file_pi:
  pickle.dump(history_modelResnet152_gs.history, file_pi)

In [ ]:
modelResnet152_tfl.save('/content/drive/MyDrive/tfg/results/ResNet152/TFL/Normal/modelResnet152_tfl.h')
modelResnet152_tfl.save('/content/drive/MyDrive/tfg/results/ResNet152/TFL/Normal/modelResnet152_tfl.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet152/TFL/Normal/modelResnet152_tfl', 'wb') as file_pi:
  pickle.dump(history_modelResnet152_tfl.history, file_pi)

In [ ]:
modelResnet50V2_rgb.save('/content/drive/MyDrive/tfg/results/ResNet50V2/RGB/Normal/modelResnet50V2_rgb.h')
modelResnet50V2_rgb.save('/content/drive/MyDrive/tfg/results/ResNet50V2/RGB/Normal/modelResnet50V2_rgb.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet50V2/RGB/Normal/modelResnet50V2_rgb', 'wb') as file_pi:
  pickle.dump(history_modelResnet50V2_rgb.history, file_pi)

In [ ]:
modelResnet50V2_gs.save('/content/drive/MyDrive/tfg/results/ResNet50V2/GS/Normal/modelResnet50V2_gs.h')
modelResnet50V2_gs.save('/content/drive/MyDrive/tfg/results/ResNet50V2/GS/Normal/modelResnet50V2_gs.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet50V2/GS/Normal/modelResnet50V2_gs', 'wb') as file_pi:
  pickle.dump(history_modelResnet50V2_gs.history, file_pi)

In [ ]:
modelResnet50V2_tfl.save('/content/drive/MyDrive/tfg/results/ResNet50V2/TFL/Normal/modelResnet50V2_tfl.h')
modelResnet50V2_tfl.save('/content/drive/MyDrive/tfg/results/ResNet50V2/TFL/Normal/modelResnet50V2_tfl.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet50V2/TFL/Normal/modelResnet50V2_tfl', 'wb') as file_pi:
  pickle.dump(history_modelResnet50V2_tfl.history, file_pi)

In [ ]:
modelResnet101V2_rgb.save('/content/drive/MyDrive/tfg/results/ResNet101V2/RGB/Normal/modelResnet101V2_rgb.h')
modelResnet101V2_rgb.save('/content/drive/MyDrive/tfg/results/ResNet101V2/RGB/Normal/modelResnet101V2_rgb.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet101V2/RGB/Normal/modelResnet101V2_rgb', 'wb') as file_pi:
  pickle.dump(history_modelResnet101V2_rgb.history, file_pi)

In [ ]:
modelResnet101V2_gs.save('/content/drive/MyDrive/tfg/results/ResNet101V2/GS/Normal/modelResnet101V2_gs.h')
modelResnet101V2_gs.save('/content/drive/MyDrive/tfg/results/ResNet101V2/GS/Normal/modelResnet101V2_gs.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet101V2/GS/Normal/modelResnet101V2_gs', 'wb') as file_pi:
  pickle.dump(history_modelResnet101V2_gs.history, file_pi)

In [ ]:
modelResnet101V2_tfl.save('/content/drive/MyDrive/tfg/results/ResNet101V2/TFL/Normal/modelResnet101V2_tfl.h')
modelResnet101V2_tfl.save('/content/drive/MyDrive/tfg/results/ResNet101V2/TFL/Normal/modelResnet101V2_tfl.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet101V2/TFL/Normal/modelResnet101V2_tfl', 'wb') as file_pi:
  pickle.dump(history_modelResnet101V2_tfl.history, file_pi)

In [ ]:
modelResnet152V2_rgb.save('/content/drive/MyDrive/tfg/results/ResNet152V2/RGB/Normal/modelResnet152V2_rgb.h')
modelResnet152V2_rgb.save('/content/drive/MyDrive/tfg/results/ResNet152V2/RGB/Normal/modelResnet152V2_rgb.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet152V2/RGB/Normal/modelResnet152V2_rgb', 'wb') as file_pi:
  pickle.dump(history_modelResnet152V2_rgb.history, file_pi)

In [ ]:
modelResnet152V2_gs.save('/content/drive/MyDrive/tfg/results/ResNet152V2/GS/Normal/modelResnet152V2_gs.h')
modelResnet152V2_gs.save('/content/drive/MyDrive/tfg/results/ResNet152V2/GS/Normal/modelResnet152V2_gs.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet152V2/GS/Normal/modelResnet152V2_gs', 'wb') as file_pi:
  pickle.dump(history_modelResnet152V2_gs.history, file_pi)

In [ ]:
modelResnet152V2_tfl.save('/content/drive/MyDrive/tfg/results/ResNet152V2/TFL/Normal/modelResnet152V2_tfl.h')
modelResnet152V2_tfl.save('/content/drive/MyDrive/tfg/results/ResNet152V2/TFL/Normal/modelResnet152V2_tfl.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet152V2/TFL/Normal/modelResnet152V2_tfl', 'wb') as file_pi:
  pickle.dump(history_modelResnet152V2_tfl.history, file_pi)

##Save Models With Data Augmentation

In [ ]:
modelCNN_rgb_da.save('/content/drive/MyDrive/tfg/results/CNN/RGB/DataAugmentation/modelCNN_rgb_da.h')
modelCNN_rgb_da.save('/content/drive/MyDrive/tfg/results/CNN/RGB/DataAugmentation/modelCNN_rgb_da.h5')
with open('/content/drive/MyDrive/tfg/results/CNN/RGB/DataAugmentation/modelCNN_rgb_da', 'wb') as file_pi:
  pickle.dump(history_modelCNN_rgb_da.history, file_pi)

In [ ]:
modelCNN_gs_da.save('/content/drive/MyDrive/tfg/results/CNN/GS/DataAugmentation/modelCNN_gs_da.h')
modelCNN_gs_da.save('/content/drive/MyDrive/tfg/results/CNN/GS/DataAugmentation/modelCNN_gs_da.h5')
with open('/content/drive/MyDrive/tfg/results/CNN/GS/DataAugmentation/modelCNN_gs_da', 'wb') as file_pi:
  pickle.dump(history_modelCNN_gs_da.history, file_pi)

In [ ]:
modelVGG16_rgb_da.save('/content/drive/MyDrive/tfg/results/VGG16/RGB/DataAugmentation/modelVGG16_rgb_da.h')
modelVGG16_rgb_da.save('/content/drive/MyDrive/tfg/results/VGG16/RGB/DataAugmentation/modelVGG16_rgb_da.h5')
with open('/content/drive/MyDrive/tfg/results/VGG16/RGB/DataAugmentation/modelVGG16_rgb_da', 'wb') as file_pi:
  pickle.dump(history_modelVGG16_rgb_da.history, file_pi)

In [ ]:
modelVGG16_gs_da.save('/content/drive/MyDrive/tfg/results/VGG16/GS/DataAugmentation/modelVGG16_gs_da.h')
modelVGG16_gs_da.save('/content/drive/MyDrive/tfg/results/VGG16/GS/DataAugmentation/modelVGG16_gs_da.h5')
with open('/content/drive/MyDrive/tfg/results/VGG16/GS/DataAugmentation/modelVGG16_gs_da', 'wb') as file_pi:
  pickle.dump(history_modelVGG16_gs_da.history, file_pi)

In [ ]:
modelVGG16_tfl_da.save('/content/drive/MyDrive/tfg/results/VGG16/TFL/DataAugmentation/modelVGG16_tfl_da.h')
modelVGG16_tfl_da.save('/content/drive/MyDrive/tfg/results/VGG16/TFL/DataAugmentation/modelVGG16_tfl_da.h5')
with open('/content/drive/MyDrive/tfg/results/VGG16/TFL/DataAugmentation/modelVGG16_tfl_da', 'wb') as file_pi:
  pickle.dump(history_modelVGG16_tfl_da.history, file_pi)

In [ ]:
modelVGG19_rgb_da.save('/content/drive/MyDrive/tfg/results/VGG19/RGB/DataAugmentation/modelVGG19_rgb_da.h')
modelVGG19_rgb_da.save('/content/drive/MyDrive/tfg/results/VGG19/RGB/DataAugmentation/modelVGG19_rgb_da.h5')
with open('/content/drive/MyDrive/tfg/results/VGG19/RGB/DataAugmentation/modelVGG19_rgb_da', 'wb') as file_pi:
  pickle.dump(history_modelVGG19_rgb_da.history, file_pi)

In [ ]:
modelVGG19_gs_da.save('/content/drive/MyDrive/tfg/results/VGG19/GS/DataAugmentation/modelVGG19_gs_da.h')
modelVGG19_gs_da.save('/content/drive/MyDrive/tfg/results/VGG19/GS/DataAugmentation/modelVGG19_gs_da.h5')
with open('/content/drive/MyDrive/tfg/results/VGG19/GS/DataAugmentation/modelVGG19_gs_da', 'wb') as file_pi:
  pickle.dump(history_modelVGG19_gs_da.history, file_pi)

In [ ]:
modelVGG19_tfl_da.save('/content/drive/MyDrive/tfg/results/VGG19/TFL/DataAugmentation/modelVGG19_tfl_da.h')
modelVGG19_tfl_da.save('/content/drive/MyDrive/tfg/results/VGG19/TFL/DataAugmentation/modelVGG19_tfl_da.h5')
with open('/content/drive/MyDrive/tfg/results/VGG19/TFL/DataAugmentation/modelVGG19_tfl_da', 'wb') as file_pi:
  pickle.dump(history_modelVGG19_tfl_da.history, file_pi)

In [ ]:
modelXception_rgb_da.save('/content/drive/MyDrive/tfg/results/Xception/RGB/DataAugmentation/modelXception_rgb_da.h')
modelXception_rgb_da.save('/content/drive/MyDrive/tfg/results/Xception/RGB/DataAugmentation/modelXception_rgb_da.h5')
with open('/content/drive/MyDrive/tfg/results/Xception/RGB/DataAugmentation/modelXception_rgb_da', 'wb') as file_pi:
  pickle.dump(history_modelXception_rgb_da.history, file_pi)

In [ ]:
modelXception_tfl_da.save('/content/drive/MyDrive/tfg/results/Xception/TFL/DataAugmentation/modelXception_tfl_da.h')
modelXception_tfl_da.save('/content/drive/MyDrive/tfg/results/Xception/TFL/DataAugmentation/modelXception_tfl_da.h5')
with open('/content/drive/MyDrive/tfg/results/Xception/TFL/DataAugmentation/modelXception_tfl_da', 'wb') as file_pi:
  pickle.dump(history_modelXception_tfl_da.history, file_pi)

In [ ]:
modelXception_gs_da.save('/content/drive/MyDrive/tfg/results/Xception/GS/DataAugmentation/modelXception_gs_da.h')
modelXception_gs_da.save('/content/drive/MyDrive/tfg/results/Xception/GS/DataAugmentation/modelXception_gs_da.h5')
with open('/content/drive/MyDrive/tfg/results/Xception/GS/DataAugmentation/modelXception_gs_da', 'wb') as file_pi:
  pickle.dump(history_modelXception_gs_da.history, file_pi)

In [ ]:
modelResnet50_rgb_da.save('/content/drive/MyDrive/tfg/results/ResNet50/RGB/DataAugmentation/modelResnet50_rgb_da.h')
modelResnet50_rgb_da.save('/content/drive/MyDrive/tfg/results/ResNet50/RGB/DataAugmentation/modelResnet50_rgb_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet50/RGB/DataAugmentation/modelResnet50_rgb_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet50_rgb_da.history, file_pi)

In [ ]:
modelResnet50_gs_da.save('/content/drive/MyDrive/tfg/results/ResNet50/GS/DataAugmentation/modelResnet50_gs_da.h')
modelResnet50_gs_da.save('/content/drive/MyDrive/tfg/results/ResNet50/GS/DataAugmentation/modelResnet50_gs_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet50/GS/DataAugmentation/modelResnet50_gs_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet50_gs_da.history, file_pi)

In [ ]:
modelResnet50_tfl_da.save('/content/drive/MyDrive/tfg/results/ResNet50/TFL/DataAugmentation/modelResnet50_tfl_da.h')
modelResnet50_tfl_da.save('/content/drive/MyDrive/tfg/results/ResNet50/TFL/DataAugmentation/modelResnet50_tfl_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet50/TFL/DataAugmentation/modelResnet50_tfl_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet50_tfl_da.history, file_pi)

In [ ]:
modelResnet101_rgb_da.save('/content/drive/MyDrive/tfg/results/ResNet101/RGB/DataAugmentation/modelResnet101_rgb_da.h')
modelResnet101_rgb_da.save('/content/drive/MyDrive/tfg/results/ResNet101/RGB/DataAugmentation/modelResnet101_rgb_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet101/RGB/DataAugmentation/modelResnet101_rgb_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet101_rgb_da.history, file_pi)

In [ ]:
modelResnet101_gs_da.save('/content/drive/MyDrive/tfg/results/ResNet101/GS/DataAugmentation/modelResnet101_gs_da.h')
modelResnet101_gs_da.save('/content/drive/MyDrive/tfg/results/ResNet101/GS/DataAugmentation/modelResnet101_gs_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet101/GS/DataAugmentation/modelResnet101_gs_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet101_gs_da.history, file_pi)

In [ ]:
modelResnet101_tfl_da.save('/content/drive/MyDrive/tfg/results/ResNet101/TFL/DataAugmentation/modelResnet101_tfl_da.h')
modelResnet101_tfl_da.save('/content/drive/MyDrive/tfg/results/ResNet101/TFL/DataAugmentation/modelResnet101_tfl_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet101/TFL/DataAugmentation/modelResnet101_tfl_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet101_tfl_da.history, file_pi)

In [ ]:
modelResnet152_rgb_da.save('/content/drive/MyDrive/tfg/results/ResNet152/RGB/DataAugmentation/modelResnet152_rgb_da.h')
modelResnet152_rgb_da.save('/content/drive/MyDrive/tfg/results/ResNet152/RGB/DataAugmentation/modelResnet152_rgb_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet152/RGB/DataAugmentation/modelResnet152_rgb_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet152_rgb_da.history, file_pi)

In [ ]:
modelResnet152_gs_da.save('/content/drive/MyDrive/tfg/results/ResNet152/GS/DataAugmentation/modelResnet152_gs_da.h')
modelResnet152_gs_da.save('/content/drive/MyDrive/tfg/results/ResNet152/GS/DataAugmentation/modelResnet152_gs_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet152/GS/DataAugmentation/modelResnet152_gs_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet152_gs_da.history, file_pi)

In [ ]:
modelResnet152_tfl_da.save('/content/drive/MyDrive/tfg/results/ResNet152/TFL/DataAugmentation/modelResnet152_tfl_da.h')
modelResnet152_tfl_da.save('/content/drive/MyDrive/tfg/results/ResNet152/TFL/DataAugmentation/modelResnet152_tfl_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet152/TFL/DataAugmentation/modelResnet152_tfl_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet152_tfl_da.history, file_pi)

In [ ]:
modelResnet50V2_rgb_da.save('/content/drive/MyDrive/tfg/results/ResNet50V2/RGB/DataAugmentation/modelResnet50V2_rgb_da.h')
modelResnet50V2_rgb_da.save('/content/drive/MyDrive/tfg/results/ResNet50V2/RGB/DataAugmentation/modelResnet50V2_rgb_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet50V2/RGB/DataAugmentation/modelResnet50V2_rgb_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet50V2_rgb_da.history, file_pi)

In [ ]:
modelResnet50V2_gs_da.save('/content/drive/MyDrive/tfg/results/ResNet50V2/GS/DataAugmentation/modelResnet50V2_gs_da.h')
modelResnet50V2_gs_da.save('/content/drive/MyDrive/tfg/results/ResNet50V2/GS/DataAugmentation/modelResnet50V2_gs_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet50V2/GS/DataAugmentation/modelResnet50V2_gs_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet50V2_gs_da.history, file_pi)

In [ ]:
modelResnet50V2_tfl_da.save('/content/drive/MyDrive/tfg/results/ResNet50V2/TFL/DataAugmentation/modelResnet50V2_tfl_da.h')
modelResnet50V2_tfl_da.save('/content/drive/MyDrive/tfg/results/ResNet50V2/TFL/DataAugmentation/modelResnet50V2_tfl_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet50V2/TFL/DataAugmentation/modelResnet50V2_tfl_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet50V2_tfl_da.history, file_pi)

In [ ]:
modelResnet101V2_rgb_da.save('/content/drive/MyDrive/tfg/results/ResNet101V2/RGB/DataAugmentation/modelResnet101V2_rgb_da.h')
modelResnet101V2_rgb_da.save('/content/drive/MyDrive/tfg/results/ResNet101V2/RGB/DataAugmentation/modelResnet101V2_rgb_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet101V2/RGB/DataAugmentation/modelResnet101V2_rgb_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet101V2_rgb_da.history, file_pi)

In [ ]:
modelResnet101V2_gs_da.save('/content/drive/MyDrive/tfg/results/ResNet101V2/GS/DataAugmentation/modelResnet101V2_gs_da.h')
modelResnet101V2_gs_da.save('/content/drive/MyDrive/tfg/results/ResNet101V2/GS/DataAugmentation/modelResnet101V2_gs_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet101V2/GS/DataAugmentation/modelResnet101V2_gs_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet101V2_gs_da.history, file_pi)

In [ ]:
modelResnet101V2_tfl_da.save('/content/drive/MyDrive/tfg/results/ResNet101V2/TFL/DataAugmentation/modelResnet101V2_tfl_da.h')
modelResnet101V2_tfl_da.save('/content/drive/MyDrive/tfg/results/ResNet101V2/TFL/DataAugmentation/modelResnet101V2_tfl_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet101V2/TFL/DataAugmentation/modelResnet101V2_tfl_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet101V2_tfl_da.history, file_pi)

In [ ]:
modelResnet152V2_rgb_da.save('/content/drive/MyDrive/tfg/results/ResNet152V2/RGB/DataAugmentation/modelResnet152V2_rgb_da.h')
modelResnet152V2_rgb_da.save('/content/drive/MyDrive/tfg/results/ResNet152V2/RGB/DataAugmentation/modelResnet152V2_rgb_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet152V2/RGB/DataAugmentation/modelResnet152V2_rgb_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet152V2_rgb_da.history, file_pi)

In [ ]:
modelResnet152V2_gs_da.save('/content/drive/MyDrive/tfg/results/ResNet152V2/GS/DataAugmentation/modelResnet152V2_gs_da.h')
modelResnet152V2_gs_da.save('/content/drive/MyDrive/tfg/results/ResNet152V2/GS/DataAugmentation/modelResnet152V2_gs_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet152V2/GS/DataAugmentation/modelResnet152V2_gs_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet152V2_gs_da.history, file_pi)

In [ ]:
modelResnet152V2_tfl_da.save('/content/drive/MyDrive/tfg/results/ResNet152V2/TFL/DataAugmentation/modelResnet152V2_tfl_da.h')
modelResnet152V2_tfl_da.save('/content/drive/MyDrive/tfg/results/ResNet152V2/TFL/DataAugmentation/modelResnet152V2_tfl_da.h5')
with open('/content/drive/MyDrive/tfg/results/ResNet152V2/TFL/DataAugmentation/modelResnet152V2_tfl_da', 'wb') as file_pi:
  pickle.dump(history_modelResnet152V2_tfl_da.history, file_pi)